# Import Library

In [1]:
import pandas as pd
import numpy as np
import re
from pymongo import MongoClient
import sys
import os
import ast
from tabulate import tabulate
import torch

code_path = os.path.abspath('../')
sys.path.append(code_path)

from credentials import connection_string

In [2]:
database_name = "Negara"

# Import Data

## Connect to MongoDB Cluster

In [3]:
collection_name = f"Bali_{database_name}_Raw"

In [4]:
client = MongoClient(connection_string)
db = client[database_name]
collection = db[collection_name]

## Get All Documents from Selected Collection

In [5]:
all_documents = collection.find()

## Convert to DataFrame

In [6]:
df = pd.DataFrame(list(all_documents))

# drop _id
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

In [7]:
df

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,94/Pid.B/2024/PN Nga,Pencurian,"Rabu, 21 Agu. 2024",29 Hari,[SUDARMAN],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...","[Ida Bagus Made Ari Suamba~Hakim Ketua~Tidak, ...",5,[SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bul...,MENGADILI:\n1. Menyatakan Terdakwa Sudarman te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,82/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[FATHULLAH als. ATUL],"[Ni Wayan Iustikasari,S.H., Selma Nabillah,S.H.]","[Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya, Reg...",3,[FATHULLAH als. ATUL~Pidana Penjara Waktu Tert...,MENGADILI:\nMenyatakan Terdakwa FATHULLAH Alia...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,83/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[I MADE SUDI ARTANA YASA],"[Delfi Trimariono, S.H., Kadek Cintyadewi Perm...","[Ni Gusti Made Utami~Hakim Ketua~Ya, Gde Putu ...",3,[I MADE SUDI ARTANA YASA~Pidana Penjara Waktu ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudiart...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-523/JEMBR...
3,Minutasi,78/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[HEPY HARDIANTO PURBA],[Lailani Rahma Indah],"[Ida Bagus Made Ari Suamba~Hakim Ketua~Ya, Ind...",5,[HEPY HARDIANTO PURBA~Pidana Penjara Waktu Ter...,MENGADILI:\nMenyatakan Terdakwa Hepy Hardianto...,“Demi Keadilan Dan Kebenaran\nBerdasarkan Ketu...
4,Minutasi,79/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[DWIE SULISTYANTO AZIS PURBOCAHYONO],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...","[Ida Bagus Made Ari Suamba~Hakim Ketua~Ya, Ind...",5,[DWIE SULISTYANTO AZIS PURBOCAHYONO~Pidana Pen...,MENGADILI:\nMenyatakan Terdakwa Dwie Sulistyan...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-513/N.1.1...
...,...,...,...,...,...,...,...,...,...,...,...,...
620,Minutasi,104/Pid.Sus/2017/PN Nga,Lalu Lintas,"Kamis, 05 Okt. 2017",53 Hari,[RIZKY AUFARRIGA HAJIZAR ZULMY],"[HANIF HARTADI, SH.]","[Fakhrudin Said Ngaji~Hakim Ketua~Ya, Mohammad...",2,[RIZKY AUFARRIGA HAJIZAR ZULMY~Pidana Penjara ...,Menyatakan Terdakwa RIZKY AUFARRIGA HAJIZAR ZU...,DAKWAAN :\n------ Bahwa terdakwa RIZKY AUFARRI...
621,Minutasi,98/Pid.Sus/2017/PN Nga,Narkotika,"Kamis, 28 Sep. 2017",53 Hari,[SAFI'I],"[AKHIRUDIN VAMI KEMALSA, SH]","[Fakhrudin Said Ngaji~Hakim Ketua~Ya, Mohammad...",3,[SAFI'I~Pidana Penjara Waktu Tertentu (15 Tahu...,Menyatakan Terdakwa SAFI”I telah terbukti seca...,KESATU\nBahwa terdakwa SAFI’I pada hari Jumat ...
622,Minutasi,99/Pid.B/2017/PN Nga,Penggelapan,"Kamis, 28 Sep. 2017",47 Hari,[NI KOMANG AYU SRI KUSNELY],"[Ni Wayan Deasy Sriaryani, SH.]","[R.R. Diah Poernomojekti~Hakim Ketua~Ya, Moham...",4,[NI KOMANG AYU SRI KUSNELY~Pidana Penjara Wakt...,Menyatakan Terdakwa NI KOMANG AYU SRI KUSNELY ...,"Bahwa ia terdakwa NI KOMANG AYU SRI KUSNELY, s..."
623,Minutasi,96/Pid.B/2017/PN Nga,Penggelapan,"Senin, 18 Sep. 2017",50 Hari,[I PUTU GERIA],"[I MADE GDE BAMAXS WIRA WIBOWO, SH.]","[R.R. Diah Poernomojekti~Hakim Ketua~Ya, Moham...",2,[I PUTU GERIA~Pidana Penjara Waktu Tertentu (5...,M E N G A D I L I\nMenyatakan Terdakwa I PUTU ...,Bahwa terdakwa I PUTU GERIA mulai pada bulan P...


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 625 entries, 0 to 624
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   status_perkara       625 non-null    object
 1   nomor_perkara        625 non-null    object
 2   klasifikasi_perkara  625 non-null    object
 3   tanggal_pendaftaran  625 non-null    object
 4   lama_proses          625 non-null    object
 5   terdakwa             625 non-null    object
 6   penuntut_umum        625 non-null    object
 7   hakim                625 non-null    object
 8   jumlah_saksi         625 non-null    int64 
 9   putusan_hukuman      625 non-null    object
 10  barang_bukti         625 non-null    object
 11  dakwaan              625 non-null    object
dtypes: int64(1), object(11)
memory usage: 58.7+ KB


# Data Understanding

In [9]:
df.head(5)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan
0,Minutasi,94/Pid.B/2024/PN Nga,Pencurian,"Rabu, 21 Agu. 2024",29 Hari,[SUDARMAN],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...","[Ida Bagus Made Ari Suamba~Hakim Ketua~Tidak, ...",5,[SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bul...,MENGADILI:\n1. Menyatakan Terdakwa Sudarman te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,82/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[FATHULLAH als. ATUL],"[Ni Wayan Iustikasari,S.H., Selma Nabillah,S.H.]","[Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya, Reg...",3,[FATHULLAH als. ATUL~Pidana Penjara Waktu Tert...,MENGADILI:\nMenyatakan Terdakwa FATHULLAH Alia...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,83/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[I MADE SUDI ARTANA YASA],"[Delfi Trimariono, S.H., Kadek Cintyadewi Perm...","[Ni Gusti Made Utami~Hakim Ketua~Ya, Gde Putu ...",3,[I MADE SUDI ARTANA YASA~Pidana Penjara Waktu ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudiart...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-523/JEMBR...
3,Minutasi,78/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[HEPY HARDIANTO PURBA],[Lailani Rahma Indah],"[Ida Bagus Made Ari Suamba~Hakim Ketua~Ya, Ind...",5,[HEPY HARDIANTO PURBA~Pidana Penjara Waktu Ter...,MENGADILI:\nMenyatakan Terdakwa Hepy Hardianto...,“Demi Keadilan Dan Kebenaran\nBerdasarkan Ketu...
4,Minutasi,79/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[DWIE SULISTYANTO AZIS PURBOCAHYONO],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...","[Ida Bagus Made Ari Suamba~Hakim Ketua~Ya, Ind...",5,[DWIE SULISTYANTO AZIS PURBOCAHYONO~Pidana Pen...,MENGADILI:\nMenyatakan Terdakwa Dwie Sulistyan...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-513/N.1.1...


**Dataset Features**
| Fitur | Deskripsi | Tipe Data |
|-----------------------|-----------------------------------------------------------------------|:------------:|
| `status_perkara` | Status dari perkara.| `str` |
| `nomor_perkara` | Nomor identifikasi unik untuk perkara.| `str` |
| `klasifikasi_perkara` | Kategori atau jenis perkara.| `str` |
| `tanggal_pendaftaran` | Tanggal ketika perkara didaftarkan di pengadilan.| `str` |
| `lama_proses` | Durasi waktu yang dibutuhkan untuk memproses perkara.| `str` |
| `terdakwa` | Nama individu yang didakwa dalam perkara tersebut.| `list[str]` |
| `penuntut_umum` | Nama jaksa atau penuntut yang menangani perkara.| `list[str]`  |
| `hakim` | Nama hakim yang memimpin sidang perkara.| `list[str]` |
| `jumlah_saksi` | Total jumlah saksi yang dihadirkan dalam sidang.| `int` |
| `putusan_hukuman` | Putusan atau keputusan yang dijatuhkan oleh hakim.| `list[str]` |
| `barang_bukti` | Amar Putusan yang berisi tentang barang bukti perkara.| `str` |
| `dakwaan`| Uraian mengenai tuduhan atau dakwaan yang diajukan terhadap terdakwa.| `str` |







# Data Preprocessing

## Data Cleaning

### Data Cleaning: Null Values

In [10]:
null_counts = df.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


In [11]:
null_rows = df[df.isnull().any(axis=1)]

print("Rows with null values:")
print(null_rows)


Rows with null values:
Empty DataFrame
Columns: [status_perkara, nomor_perkara, klasifikasi_perkara, tanggal_pendaftaran, lama_proses, terdakwa, penuntut_umum, hakim, jumlah_saksi, putusan_hukuman, barang_bukti, dakwaan]
Index: []


In [12]:
df_copy = df.dropna()

null_counts = df_copy.isnull().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara         0
nomor_perkara          0
klasifikasi_perkara    0
tanggal_pendaftaran    0
lama_proses            0
terdakwa               0
penuntut_umum          0
hakim                  0
jumlah_saksi           0
putusan_hukuman        0
barang_bukti           0
dakwaan                0
dtype: int64


In [13]:
print(f"Jumlah data setelah null dihapus: {df_copy.shape[0]}")

Jumlah data setelah null dihapus: 625


### Data Cleaning: Data Disamarkan

#### Check Data "Disamarkan"

In [14]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 58


In [15]:
print(count_disamarkan)

    status_perkara            nomor_perkara            klasifikasi_perkara  \
7         Minutasi   68/Pid.Sus/2024/PN Nga              Perlindungan Anak   
8         Minutasi   69/Pid.Sus/2024/PN Nga              Perlindungan Anak   
9         Minutasi   70/Pid.Sus/2024/PN Nga              Perlindungan Anak   
22        Minutasi   56/Pid.Sus/2024/PN Nga                      Lain-Lain   
27        Minutasi   51/Pid.Sus/2024/PN Nga              Perlindungan Anak   
44        Minutasi   28/Pid.Sus/2024/PN Nga              Perlindungan Anak   
57        Minutasi   14/Pid.Sus/2024/PN Nga              Perlindungan Anak   
58        Minutasi   15/Pid.Sus/2024/PN Nga              Perlindungan Anak   
59        Minutasi   12/Pid.Sus/2024/PN Nga              Perlindungan Anak   
68        Minutasi    5/Pid.Sus/2024/PN Nga              Perlindungan Anak   
80        Minutasi  124/Pid.Sus/2023/PN Nga              Perlindungan Anak   
85        Minutasi  119/Pid.Sus/2023/PN Nga              Perlind

#### Drop Data "Disamarkan"

In [16]:
df_copy = df_copy[~df_copy["dakwaan"].str.lower().isin(["disamarkan"])]
df_copy.reset_index(drop=True, inplace=True)

In [17]:
count_disamarkan = df_copy[df_copy["dakwaan"].str.lower().isin(["disamarkan"])]

print(f"Jumlah data 'Disamarkan': {count_disamarkan.shape[0]}")

Jumlah data 'Disamarkan': 0


In [18]:
print(f"Jumlah data setelah disamarkan dihapus: {df_copy.shape[0]}")

Jumlah data setelah disamarkan dihapus: 567


### Data Cleaning: Status Perkara

In [19]:
unique_status_perkara_values = df_copy['status_perkara'].unique()
num_unique_status_perkara = len(unique_status_perkara_values)

print(f"Number of unique values in 'status_perkara': {num_unique_status_perkara}")
print("Unique values:", unique_status_perkara_values)


Number of unique values in 'status_perkara': 1
Unique values: ['Minutasi']


### Data Cleaning: Nomor Perkara

In [21]:
unique_nomor_perkara_values = df_copy['nomor_perkara'].unique()
num_unique_nomor_perkara = len(unique_nomor_perkara_values)

print(f"Jumlah Data: {len(df_copy)}")
print(f"Jumlah Nilai Unique pada Kolom 'nomor_perkara': {num_unique_nomor_perkara}")

Jumlah Data: 567
Jumlah Nilai Unique pada Kolom 'nomor_perkara': 567


### Data Cleaning: Klasifikasi Perkara

In [22]:
unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
print(unique_klasifikasi_counts)

Number of unique values in 'status_perkara': 29
klasifikasi_perkara
Pencurian                                                                                    174
Narkotika                                                                                     94
Kejahatan Perjudian                                                                           42
Penggelapan                                                                                   37
Lalu Lintas                                                                                   35
Lain-Lain                                                                                     31
Kesehatan                                                                                     30
Penebangan Kayu                                                                               22
Penipuan                                                                                      21
Penganiayaan                                               

### TEMP!!

In [20]:
# df_copy = df_copy[~df_copy["klasifikasi_perkara"].isin(["Lalu Lintas"])]
# df_copy.reset_index(drop=True, inplace=True)

In [21]:
# unique_klasifikasi_perkara_values = df_copy['klasifikasi_perkara'].unique()
# num_unique_klasifikasi_perkara = len(unique_klasifikasi_perkara_values)
# unique_klasifikasi_counts = df_copy['klasifikasi_perkara'].value_counts()

# print(f"Number of unique values in 'status_perkara': {num_unique_klasifikasi_perkara}")
# print(unique_klasifikasi_counts)

### Data Cleaning: Hakim

#### Check "Hakim" Column Data Type (Expected List)

In [23]:
unique_types = df_copy['hakim'].apply(type).unique()
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


In [24]:
count_str_hakim = df_copy[df_copy["hakim"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'hakim' berupa string: {count_str_hakim}")

Jumlah data dengan 'hakim' berupa string: 0


#### Convert String to Original Data Type (List)

In [25]:
for index, row in df_copy.iterrows():
    value = row["hakim"]
    
    if isinstance(value, str):
        try:
            print(f"Type: {type(value)}, Row {index} - Value: {value}")
            value = ast.literal_eval(value) 
            df_copy.at[index, "hakim"] = value 
            print(f"Type: {type(value)}, Row {index} - Value: {value}\n")
            
        except (ValueError, SyntaxError):
            print(f"Error in Row {index}: Not a valid list format.")
    

In [26]:
unique_types = df_copy['hakim'].apply(type).unique()
print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


#### Check Unique Values of Total "Hakim"

In [27]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[ 6  3  9 12  0]


In [28]:
df_copy = df_copy[df_copy['hakim'].apply(len) > 0]
df_copy.reset_index(drop=True, inplace=True)

In [29]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[ 6  3  9 12]


In [30]:
for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 6: 
        print(f"{index} -> {row['hakim']}")

0 -> ['Ida Bagus Made Ari Suamba~Hakim Ketua~Tidak', 'Indah Wahyuni Dian Ratnasari~Hakim Anggota~Tidak', 'Nanda Riwanto~Hakim Anggota~Tidak', 'Ida Bagus Made Ari Suamba~Hakim Ketua~Ya', 'Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya', 'Regy Trihardianto~Hakim Anggota~Ya']
6 -> ['Satriyo Murtitomo~Hakim Ketua~Tidak', 'Gde Putu Oka Yoga Bharata~Hakim Anggota~Tidak', 'Wajihatut Dzikriyah~Hakim Anggota~Tidak', 'Satriyo Murtitomo~Hakim Ketua~Ya', 'Gde Putu Oka Yoga Bharata~Hakim Anggota~Ya', 'Aziz Junaedi~Hakim Anggota~Ya']
8 -> ['Gde Putu Oka Yoga Bharata~Hakim Ketua~Tidak', 'Satriyo Murtitomo~Hakim Anggota~Tidak', 'Wajihatut Dzikriyah~Hakim Anggota~Tidak', 'Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya', 'Satriyo Murtitomo~Hakim Anggota~Ya', 'Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya']
12 -> ['Gde Putu Oka Yoga Bharata~Hakim Ketua~Tidak', 'Satriyo Murtitomo~Hakim Anggota~Tidak', 'Wajihatut Dzikriyah~Hakim Anggota~Tidak', 'Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya', 'Satriyo Murtitomo~Hakim 

In [31]:
for index, row in df_copy.iterrows():
    if len(row["hakim"]) == 9: 
        print(f"{index} -> {row['hakim']}")

7 -> ['Satriyo Murtitomo~Hakim Ketua~Tidak', 'Gde Putu Oka Yoga Bharata~Hakim Anggota~Tidak', 'Wajihatut Dzikriyah~Hakim Anggota~Tidak', 'Satriyo Murtitomo~Hakim Ketua~Tidak', 'Gde Putu Oka Yoga Bharata~Hakim Anggota~Tidak', 'Nanda Riwanto~Hakim Anggota~Tidak', 'Satriyo Murtitomo~Hakim Ketua~Ya', 'Gde Putu Oka Yoga Bharata~Hakim Anggota~Ya', 'Aziz Junaedi~Hakim Anggota~Ya']
9 -> ['Indah Wahyuni Dian Ratnasari~Hakim Ketua~Tidak', 'Nanda Riwanto~Hakim Anggota~Tidak', 'Wajihatut Dzikriyah~Hakim Anggota~Tidak', 'Indah Wahyuni Dian Ratnasari~Hakim Ketua~Tidak', 'Nanda Riwanto~Hakim Anggota~Tidak', 'Aziz Junaedi~Hakim Anggota~Tidak', 'Indah Wahyuni Dian Ratnasari~Hakim Ketua~Ya', 'Regy Trihardianto~Hakim Anggota~Ya', 'Aziz Junaedi~Hakim Anggota~Ya']
16 -> ['Ni Kadek Kusuma Wardani~Hakim Ketua~Tidak', 'Satriyo Murtitomo~Hakim Anggota~Tidak', 'Wajihatut Dzikriyah~Hakim Anggota~Tidak', 'Satriyo Murtitomo~Hakim Ketua~Tidak', 'Gde Putu Oka Yoga Bharata~Hakim Anggota~Tidak', 'Indah Wahyuni Dian Ra

In [32]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

Ida Bagus Made Ari Suamba~Hakim Ketua~Tidak
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Tidak
Nanda Riwanto~Hakim Anggota~Tidak
Ida Bagus Made Ari Suamba~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Regy Trihardianto~Hakim Anggota~Ya

Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya
Regy Trihardianto~Hakim Anggota~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya

Ni Gusti Made Utami~Hakim Ketua~Ya
Gde Putu Oka Yoga Bharata~Hakim Anggota~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya

Ida Bagus Made Ari Suamba~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Nanda Riwanto~Hakim Anggota~Ya

Ida Bagus Made Ari Suamba~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Nanda Riwanto~Hakim Anggota~Ya

Ni Gusti Made Utami~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Nanda Riwanto~Hakim Anggota~Ya

Satriyo Murtitomo~Hakim Ketua~Tidak
Gde Putu Oka Yoga Bharata~Hakim Anggota~Tidak
Wajihatut Dzikriyah~Hakim Anggota~Tidak
Satriyo Murtitomo~Hakim Ketua

#### Remove Hakim Tidak Aktif

In [33]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [entry for entry in x if '~Tidak' not in entry])

In [34]:
for index, row in df_copy.iterrows():
    i = len(row["hakim"])
    for j in range(i):
        print(row["hakim"][j])
    print("")

Ida Bagus Made Ari Suamba~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Regy Trihardianto~Hakim Anggota~Ya

Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya
Regy Trihardianto~Hakim Anggota~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya

Ni Gusti Made Utami~Hakim Ketua~Ya
Gde Putu Oka Yoga Bharata~Hakim Anggota~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya

Ida Bagus Made Ari Suamba~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Nanda Riwanto~Hakim Anggota~Ya

Ida Bagus Made Ari Suamba~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Nanda Riwanto~Hakim Anggota~Ya

Ni Gusti Made Utami~Hakim Ketua~Ya
Indah Wahyuni Dian Ratnasari~Hakim Anggota~Ya
Nanda Riwanto~Hakim Anggota~Ya

Satriyo Murtitomo~Hakim Ketua~Ya
Gde Putu Oka Yoga Bharata~Hakim Anggota~Ya
Aziz Junaedi~Hakim Anggota~Ya

Satriyo Murtitomo~Hakim Ketua~Ya
Gde Putu Oka Yoga Bharata~Hakim Anggota~Ya
Aziz Junaedi~Hakim Anggota~Ya

Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya
Satriyo Murtitomo~Hakim

#### Check Unique Values after Removing "Hakim Tidak Aktif"

In [35]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[3]


#### Remove Hakim Anggota

In [36]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(lambda x: [value for value in x if '~Hakim Anggota' not in value])

In [37]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['Ida Bagus Made Ari Suamba~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Ida Bagus Made Ari Suamba~Hakim Ketua~Ya']
['Ida Bagus Made Ari Suamba~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Satriyo Murtitomo~Hakim Ketua~Ya']
['Satriyo Murtitomo~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Indah Wahyuni Dian Ratnasari~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Satriyo Murtitomo~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Ni Gusti Made Utami~Hakim Ketua~Ya']
['Satriyo Murtitomo~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketua~Ya']
['Ni Kadek Kusuma Wardani~Hakim Ketua~Ya']
['Gde Putu Oka Yoga Bharata~Hakim Ketu

#### Check Unique Values after Removing "Hakim Anggota" (Expected 1)

In [38]:
unique_types = df_copy['hakim'].apply(len).unique()
print(unique_types)

[1]


#### Extract Hakim Name

In [39]:
df_copy.loc[:, 'hakim'] = df_copy['hakim'].apply(
    lambda x: [value.split('~Hakim')[0].strip() for value in x]
)

In [40]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

['Ida Bagus Made Ari Suamba']
['Gde Putu Oka Yoga Bharata']
['Ni Gusti Made Utami']
['Ida Bagus Made Ari Suamba']
['Ida Bagus Made Ari Suamba']
['Ni Gusti Made Utami']
['Satriyo Murtitomo']
['Satriyo Murtitomo']
['Gde Putu Oka Yoga Bharata']
['Indah Wahyuni Dian Ratnasari']
['Ni Gusti Made Utami']
['Ni Gusti Made Utami']
['Gde Putu Oka Yoga Bharata']
['Ni Gusti Made Utami']
['Gde Putu Oka Yoga Bharata']
['Gde Putu Oka Yoga Bharata']
['Satriyo Murtitomo']
['Ni Gusti Made Utami']
['Ni Gusti Made Utami']
['Satriyo Murtitomo']
['Gde Putu Oka Yoga Bharata']
['Gde Putu Oka Yoga Bharata']
['Ni Kadek Kusuma Wardani']
['Gde Putu Oka Yoga Bharata']
['Ni Kadek Kusuma Wardani']
['Gde Putu Oka Yoga Bharata']
['Ni Kadek Kusuma Wardani']
['Gde Putu Oka Yoga Bharata']
['Ni Kadek Kusuma Wardani']
['Satriyo Murtitomo']
['Ni Gusti Made Utami']
['Ni Gusti Made Utami']
['Ni Kadek Kusuma Wardani']
['Ni Kadek Kusuma Wardani']
['Ni Kadek Kusuma Wardani']
['Ni Gusti Made Utami']
['Ni Gusti Made Utami']
['Gde P

#### Convert List to String

In [41]:
df_copy['hakim'] = df_copy['hakim'].apply(lambda x: str(x[0]) if isinstance(x, list) and len(x) > 0 else str(x))

In [42]:
unique_types = df_copy['hakim'].apply(type).unique()
print(unique_types)

[<class 'str'>]


In [43]:
for index, row in df_copy.iterrows():
    print(row['hakim'])

Ida Bagus Made Ari Suamba
Gde Putu Oka Yoga Bharata
Ni Gusti Made Utami
Ida Bagus Made Ari Suamba
Ida Bagus Made Ari Suamba
Ni Gusti Made Utami
Satriyo Murtitomo
Satriyo Murtitomo
Gde Putu Oka Yoga Bharata
Indah Wahyuni Dian Ratnasari
Ni Gusti Made Utami
Ni Gusti Made Utami
Gde Putu Oka Yoga Bharata
Ni Gusti Made Utami
Gde Putu Oka Yoga Bharata
Gde Putu Oka Yoga Bharata
Satriyo Murtitomo
Ni Gusti Made Utami
Ni Gusti Made Utami
Satriyo Murtitomo
Gde Putu Oka Yoga Bharata
Gde Putu Oka Yoga Bharata
Ni Kadek Kusuma Wardani
Gde Putu Oka Yoga Bharata
Ni Kadek Kusuma Wardani
Gde Putu Oka Yoga Bharata
Ni Kadek Kusuma Wardani
Gde Putu Oka Yoga Bharata
Ni Kadek Kusuma Wardani
Satriyo Murtitomo
Ni Gusti Made Utami
Ni Gusti Made Utami
Ni Kadek Kusuma Wardani
Ni Kadek Kusuma Wardani
Ni Kadek Kusuma Wardani
Ni Gusti Made Utami
Ni Gusti Made Utami
Gde Putu Oka Yoga Bharata
Ni Gusti Made Utami
Ni Gusti Made Utami
Ni Kadek Kusuma Wardani
Gde Putu Oka Yoga Bharata
Ni Gusti Made Utami
Ni Kadek Kusuma War

In [44]:
print(f"Jumlah data setelah proses cleaning kolom Hakim: {df_copy.shape[0]}")

Jumlah data setelah proses cleaning kolom Hakim: 564


### Data Cleaning: Barang Bukti

#### RegEx Testing on 1 Data

In [45]:
text = df_copy.iloc[-1]['barang_bukti']
print(text)

M  E  N  G  A  D  I  L  I     :
Menyatakan Terdakwa TAUFIK RAHMAN alias TAUPIQURAHMAN alias OPIK terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana “Penggelapan” sebagaimana dalam dakwaan kesatu Penuntut Umum;
Menjatuhkan pidana kepada Terdakwa tersebut oleh karena itu dengan pidana penjara selama 5 (lima) bulan;
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan;
Menetapkan Terdakwa tetap ditahan;
Menetapkan barang bukti berupa:
1 ( satu ) unit sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi : DK 2760 WO, nomor rangka : MH1KEVA134K796939, nomor mesin : KEVAE-1795825;
1 (satu) lembar STNK  atas nama I KETUT NURATA untuk sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi : DK 2760 WO, nomor rangka : MH1KEVA134K796939, nomor mesin : KEVAE-1795825;
          Dikembalikan kepada saksi An. MOCH. NASIR NIZAR
      6. Membebankan kepada Terdakwa membayar biaya p

##### Remove Unwanted Symbol and Char

In [46]:
cleaned_text = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text)
cleaned_text = re.sub(r'-{2,}', '-', cleaned_text)

In [47]:
print(cleaned_text)

M  E  N  G  A  D  I  L  I     
Menyatakan Terdakwa TAUFIK RAHMAN alias TAUPIQURAHMAN alias OPIK terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Penggelapan sebagaimana dalam dakwaan kesatu Penuntut Umum
Menjatuhkan pidana kepada Terdakwa tersebut oleh karena itu dengan pidana penjara selama 5 (lima) bulan
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan
Menetapkan Terdakwa tetap ditahan
Menetapkan barang bukti berupa
1 ( satu ) unit sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi  DK 2760 WO, nomor rangka  MH1KEVA134K796939, nomor mesin  KEVAE-1795825
1 (satu) lembar STNK  atas nama I KETUT NURATA untuk sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi  DK 2760 WO, nomor rangka  MH1KEVA134K796939, nomor mesin  KEVAE-1795825
          Dikembalikan kepada saksi An. MOCH. NASIR NIZAR
      6. Membebankan kepada Terdakwa membayar biaya perkara sejumlah 

##### Remove Multiple Whitespaces

In [48]:
cleaned_text = re.sub(r' +', ' ', cleaned_text)

In [49]:
print(cleaned_text)

M E N G A D I L I 
Menyatakan Terdakwa TAUFIK RAHMAN alias TAUPIQURAHMAN alias OPIK terbukti secara sah dan meyakinkan bersalah melakukan tindak pidana Penggelapan sebagaimana dalam dakwaan kesatu Penuntut Umum
Menjatuhkan pidana kepada Terdakwa tersebut oleh karena itu dengan pidana penjara selama 5 (lima) bulan
Menetapkan masa penangkapan dan penahanan yang telah dijalani Terdakwa dikurangkan seluruhnya dari pidana yang dijatuhkan
Menetapkan Terdakwa tetap ditahan
Menetapkan barang bukti berupa
1 ( satu ) unit sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi DK 2760 WO, nomor rangka MH1KEVA134K796939, nomor mesin KEVAE-1795825
1 (satu) lembar STNK atas nama I KETUT NURATA untuk sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi DK 2760 WO, nomor rangka MH1KEVA134K796939, nomor mesin KEVAE-1795825
 Dikembalikan kepada saksi An. MOCH. NASIR NIZAR
 6. Membebankan kepada Terdakwa membayar biaya perkara sejumlah Rp.5.000,- (lima ribu rupiah)


##### Extract Only Barang Bukti

In [50]:
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n\s*[A-Z]|$)"
# pattern = r"Menetapkan barang bukti berupa\s*(.*?)(?=\n[A-Z])"
pattern = r"menetapkan barang bukti berupa\s*(.*)"

match = re.search(pattern, cleaned_text, re.DOTALL | re.IGNORECASE)

if match:
    print(match.group().strip())
    
else:
    print("Tidak ada kecocokan ditemukan.")

Menetapkan barang bukti berupa
1 ( satu ) unit sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi DK 2760 WO, nomor rangka MH1KEVA134K796939, nomor mesin KEVAE-1795825
1 (satu) lembar STNK atas nama I KETUT NURATA untuk sepeda motor jenis Honda Supra X, tahun 2004, warna hitam, nomor polisi DK 2760 WO, nomor rangka MH1KEVA134K796939, nomor mesin KEVAE-1795825
 Dikembalikan kepada saksi An. MOCH. NASIR NIZAR
 6. Membebankan kepada Terdakwa membayar biaya perkara sejumlah Rp.5.000,- (lima ribu rupiah)


#### RegEx on Entire DataFrame

##### Extract Barang Bukti

In [51]:
# patterns = [
#     r"barang\s*bukti\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang\s*bukti,\s*berupa\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang\s*bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)",
#     r"barang-bukti\s*:?\s*(.*?)(?=\n\s*[A-Z]|$)"
# ]

patterns = [
    r"barang\s*bukti\s*berupa\s*:?\s*(.*)",
    r"barang\s*bukti,\s*berupa\s*:?\s*(.*)",
    r"barang\s*bukti\s*:?\s*(.*)",
    r"barang-bukti\s*:?\s*(.*)"
]

rows_to_drop = []

for index, row in df_copy.iterrows():
    text_barang_bukti = row['barang_bukti']
    
    cleaned_barang_bukti = re.sub(r'[^a-zA-Z0-9., /\\()\"\'\n-]+', '', text_barang_bukti)
    cleaned_barang_bukti = re.sub(r'-{2,}', '-', cleaned_barang_bukti)
    cleaned_barang_bukti = re.sub(r' +', ' ', cleaned_barang_bukti)
    
    match_found = False

    for pattern in patterns:
        match = re.search(pattern, cleaned_barang_bukti, re.DOTALL | re.IGNORECASE)

        if match:
            df_copy.loc[index, 'cleaned_barang_bukti'] = match.group(1).strip().replace('\n', ' ')
            match_found = True
            break

    
    if not match_found:
        print(f"{index}. Barang Bukti Tidak Ditemukan pada {row['nomor_perkara']}")
        rows_to_drop.append(index) 
        # print(cleaned_barang_bukti)



4. Barang Bukti Tidak Ditemukan pada 79/Pid.B/2024/PN Nga
20. Barang Bukti Tidak Ditemukan pada 53/Pid.B/2024/PN Nga
39. Barang Bukti Tidak Ditemukan pada 26/Pid.B/2024/PN Nga
41. Barang Bukti Tidak Ditemukan pada 24/Pid.B/2024/PN Nga
60. Barang Bukti Tidak Ditemukan pada 2/Pid.B/2024/PN Nga
72. Barang Bukti Tidak Ditemukan pada 122/Pid.B/2023/PN Nga
113. Barang Bukti Tidak Ditemukan pada 32/Pid.Sus/2023/PN Nga
114. Barang Bukti Tidak Ditemukan pada 33/Pid.Sus/2023/PN Nga
243. Barang Bukti Tidak Ditemukan pada 64/Pid.Sus/2021/PN Nga
381. Barang Bukti Tidak Ditemukan pada 149/Pid.B/2019/PN Nga
444. Barang Bukti Tidak Ditemukan pada 25/Pid.B/2019/PN Nga
459. Barang Bukti Tidak Ditemukan pada 6/Pid.B/2019/PN Nga
480. Barang Bukti Tidak Ditemukan pada 119/Pid.B/2018/PN Nga
527. Barang Bukti Tidak Ditemukan pada 1/Pid.B/2018/PN Nga
532. Barang Bukti Tidak Ditemukan pada 133/Pid.B/2017/PN Nga
548. Barang Bukti Tidak Ditemukan pada 109/Pid.Sus/2017/PN Nga


##### Get Case with No Barang Bukti

In [52]:
true_values = df_copy.isna()
df_fail_barang_bukti = df_copy[true_values['cleaned_barang_bukti']]

In [53]:
df_fail_barang_bukti.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
4,Minutasi,79/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[DWIE SULISTYANTO AZIS PURBOCAHYONO],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...",Ida Bagus Made Ari Suamba,5,[DWIE SULISTYANTO AZIS PURBOCAHYONO~Pidana Pen...,MENGADILI:\nMenyatakan Terdakwa Dwie Sulistyan...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-513/N.1.1...,NaN
20,Minutasi,53/Pid.B/2024/PN Nga,"Pengeroyokan yang menyebabkan luka ringan, luk...","Rabu, 05 Jun. 2024",64 Hari,"[ISMI AZIS KERAF, I MADE BAGAS ANANDRA OCTBERY...","[IDA Bagus Gede Eka Permana Putra,S.H., I Waya...",Gde Putu Oka Yoga Bharata,4,[ISMI AZIS KERAF~Pidana Penjara Waktu Tertentu...,MENGADILI:\nMenyatakan Terdakwa I ISMI AZIS KE...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,NaN
39,Minutasi,26/Pid.B/2024/PN Nga,Penganiayaan,"Rabu, 06 Mar. 2024",55 Hari,[BADRIONO],"[Ni Wayan Iustikasari,S.H., Muhammad Faisal Ar...",Ni Gusti Made Utami,2,[BADRIONO~Pidana Penjara Waktu Tertentu (6 Bul...,MENGADILI:\nMenyatakan Terdakwa Badriono terse...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN NEGERI...,NaN
41,Minutasi,24/Pid.B/2024/PN Nga,Penghancuran atau Perusakan Barang,"Selasa, 05 Mar. 2024",77 Hari,[LUKAS WIJAYA KARTA],"[Ni Made Ayu Olin,S.H., ROSSY PRASETYAWATI, SH.]",Gde Putu Oka Yoga Bharata,3,[LUKAS WIJAYA KARTA~Lain-lain],MENETAPKAN:\nMenyatakan penuntutan Penuntut Um...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,NaN
60,Minutasi,2/Pid.B/2024/PN Nga,Penganiayaan,"Rabu, 03 Jan. 2024",34 Hari,[SAPTA HADI RAHMAN],"[Ni Ketut Cahaya Listiani,S.H., Edwin Gama Pra...",Ni Kadek Kusuma Wardani,3,[SAPTA HADI RAHMAN~Pidana Penjara Waktu Terten...,MENGADILI:\nMenyatakan Terdakwa SAPTA HADI RAH...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,NaN


In [54]:
df_fail_barang_bukti.to_csv(f'../Data/Misc/{database_name}_kasus tanpa barang bukti.csv')

##### Drop Perkara Tanpa Barang Bukti

In [55]:
df_copy.drop(rows_to_drop, inplace=True)
print(f"rows dropped: {rows_to_drop}")
print(f"rows dropped: {len(rows_to_drop)}")

rows dropped: [4, 20, 39, 41, 60, 72, 113, 114, 243, 381, 444, 459, 480, 527, 532, 548]
rows dropped: 16


In [56]:
df_copy.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti
0,Minutasi,94/Pid.B/2024/PN Nga,Pencurian,"Rabu, 21 Agu. 2024",29 Hari,[SUDARMAN],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...",Ida Bagus Made Ari Suamba,5,[SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bul...,MENGADILI:\n1. Menyatakan Terdakwa Sudarman te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,- 1 (satu) unit sepeda motor Honda Scopy tahun...
1,Minutasi,82/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[FATHULLAH als. ATUL],"[Ni Wayan Iustikasari,S.H., Selma Nabillah,S.H.]",Gde Putu Oka Yoga Bharata,3,[FATHULLAH als. ATUL~Pidana Penjara Waktu Tert...,MENGADILI:\nMenyatakan Terdakwa FATHULLAH Alia...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik klip berisi kristal beni...
2,Minutasi,83/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[I MADE SUDI ARTANA YASA],"[Delfi Trimariono, S.H., Kadek Cintyadewi Perm...",Ni Gusti Made Utami,3,[I MADE SUDI ARTANA YASA~Pidana Penjara Waktu ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudiart...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-523/JEMBR...,1. 14 (empat belas) buah plastik klip berisi k...
3,Minutasi,78/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[HEPY HARDIANTO PURBA],[Lailani Rahma Indah],Ida Bagus Made Ari Suamba,5,[HEPY HARDIANTO PURBA~Pidana Penjara Waktu Ter...,MENGADILI:\nMenyatakan Terdakwa Hepy Hardianto...,“Demi Keadilan Dan Kebenaran\nBerdasarkan Ketu...,"1 (satu) unit mobil Mitsubishi Xpander, warna ..."
5,Minutasi,76/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 18 Jul. 2024",40 Hari,"[RONY SETIAWAN, DIAN HERMAWAN]","[Miranda Widyawati,S.H., Kadek Cintyadewi Perm...",Ni Gusti Made Utami,4,[RONY SETIAWAN~Pidana Penjara Waktu Tertentu (...,MENGADILI:\nMenyatakan Terdakwa I Rony Setiawa...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik berisi kristal bening na...


##### Check NaN Values

In [57]:
null_counts = df_copy.isna().sum()

print("Number of null values in each column:")
print(null_counts)

Number of null values in each column:
status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
dtype: int64


In [58]:
print(f"Jumlah data setelah proses cleaning kolom Barang Bukti: {df_copy.shape[0]}")

Jumlah data setelah proses cleaning kolom Barang Bukti: 548


### Data Cleaning: Dakwaan

#### Test Cleaning on 1 Data

In [59]:
dakwaan = df_copy.iloc[1]['dakwaan']
print(dakwaan)

KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI JEMBRANA
Jl. Udayana No. 11, Negara, Kabupaten Jembrana, 82213
Telp: (0365) 41164 Fax. (0365) 41165 www.kejari-jembrana.go.id
 “Demi Keadilan Dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa”                                                                                         P-29
  SURAT DAKWAAN
No. REG. PERKARA: PDM- 532/N.1.16/Enz.2/07/2024
  TERDAKWA:
Nama lengkap
Nomor Induk Kependudukan
:
:
FATHULAH Alias ATUL
3174031611101017
Tempat lahir
:
Loloan Barat
Umur/tanggal lahir
:
54 tahun / 01 Januari 1970
Jenis kelamin
:
Laki-laki
Kebangsaan/kewarganegaraan
:
Indonesia
Tempat tinggal
  :
  Kemang Timur Rt/Rw.010/003, Kelurahan Bangka, Kecamatan Mampang Prapatan, Kabupaten /Kota Jakarta Selatan,
Perumahan Huma Lestari Baluk, Desa Baluk, Kecamatan Negara, Kabupaten Jembrana
Agama
:
Islam
Pekerjaan
:
Buruh Harian Lepas
Pendidikan
:
SD (Tamat)
  STATUS PENANGKAPAN DAN PENAHANAN:
  Penangkapan
:
Tanggal 31 Mei 2024
  

##### Clean Unwanted Char and Symbols

In [60]:
# cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n]+', dakwaan)
# cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
# print(cleaned_dakwaan)

cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', dakwaan)

cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
cleaned_dakwaan = re.sub(r' +', ' ', cleaned_dakwaan)

print(cleaned_dakwaan)

KEJAKSAAN REPUBLIK INDONESIA
KEJAKSAAN TINGGI BALI
KEJAKSAAN NEGERI JEMBRANA
Jl. Udayana No. 11, Negara, Kabupaten Jembrana, 82213
Telp (0365) 41164 Fax. (0365) 41165 www.kejari-jembrana.go.id
 Demi Keadilan Dan Kebenaran
Berdasarkan Ketuhanan Yang Maha Esa P-29
 SURAT DAKWAAN
No. REG. PERKARA PDM- 532/N.1.16/Enz.2/07/2024
 TERDAKWA
Nama lengkap
Nomor Induk Kependudukan


FATHULAH Alias ATUL
3174031611101017
Tempat lahir

Loloan Barat
Umur/tanggal lahir

54 tahun / 01 Januari 1970
Jenis kelamin

Laki-laki
Kebangsaan/kewarganegaraan

Indonesia
Tempat tinggal
 
 Kemang Timur Rt/Rw.010/003, Kelurahan Bangka, Kecamatan Mampang Prapatan, Kabupaten /Kota Jakarta Selatan,
Perumahan Huma Lestari Baluk, Desa Baluk, Kecamatan Negara, Kabupaten Jembrana
Agama

Islam
Pekerjaan

Buruh Harian Lepas
Pendidikan

SD (Tamat)
 STATUS PENANGKAPAN DAN PENAHANAN
 Penangkapan

Tanggal 31 Mei 2024
 Penahanan
 Penyidik

Sejak Tgl 01 Juni 2024 s/d Tgl. 20 Juni 2024
 Perpanjangan PU

Sejak Tgl. 21 Juni 2024 s/d 

##### Remove Header

In [61]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

text_dakwaan = cleaned_dakwaan
matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

if matches: 
    last_match = matches[-1]
    cleaned_dakwaan = last_match.strip()
else:
    cleaned_dakwaan = text_dakwaan
    print(f"raw dakwaan copied at {index}")

print(cleaned_dakwaan)

DAKWAAN
PERTAMA
-Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut
 Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi Ilham Aris Darmawan akan be

##### Remove Footer

In [62]:
pattern = r"^(.*?)\s+Negara,\s+\d{1,2}\s+\w+\s+\d{4}"

text_dakwaan = cleaned_dakwaan
matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

if matches: 
    last_match = matches[-1]
    cleaned_dakwaan = last_match.strip()
else:
    cleaned_dakwaan = text_dakwaan
    print(f"raw dakwaan copied at {index}")

print(cleaned_dakwaan)

raw dakwaan copied at 563
DAKWAAN
PERTAMA
-Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut
 Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi I

#### Cleaning on Entire DataFrame

##### Clean Unwanted Char & Symbols

In [63]:
def clean_dakwaan(text):
    cleaning_dakwaan_unwanted_char = re.findall(r'[a-zA-Z0-9., /\\()\"\'\n-]+', text)
    cleaned_dakwaan = ''.join(cleaning_dakwaan_unwanted_char)
    
    cleaned_dakwaan = re.sub(r'-{2,}', '-', cleaned_dakwaan)
    cleaned_dakwaan = re.sub(r' +', ' ', cleaned_dakwaan)
    
    return cleaned_dakwaan

In [64]:
df_copy['cleaned_dakwaan'] = df_copy['dakwaan'].apply(clean_dakwaan)

In [65]:
df_copy.head()

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,94/Pid.B/2024/PN Nga,Pencurian,"Rabu, 21 Agu. 2024",29 Hari,[SUDARMAN],"[Lailani Rahma Indah, Kadek Cintyadewi Permana...",Ida Bagus Made Ari Suamba,5,[SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bul...,MENGADILI:\n1. Menyatakan Terdakwa Sudarman te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,- 1 (satu) unit sepeda motor Honda Scopy tahun...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
1,Minutasi,82/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[FATHULLAH als. ATUL],"[Ni Wayan Iustikasari,S.H., Selma Nabillah,S.H.]",Gde Putu Oka Yoga Bharata,3,[FATHULLAH als. ATUL~Pidana Penjara Waktu Tert...,MENGADILI:\nMenyatakan Terdakwa FATHULLAH Alia...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik klip berisi kristal beni...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...
2,Minutasi,83/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,[I MADE SUDI ARTANA YASA],"[Delfi Trimariono, S.H., Kadek Cintyadewi Perm...",Ni Gusti Made Utami,3,[I MADE SUDI ARTANA YASA~Pidana Penjara Waktu ...,MENGADILI:\nMenyatakan Terdakwa I Made Sudiart...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-523/JEMBR...,1. 14 (empat belas) buah plastik klip berisi k...,SURAT DAKWAAN\nNo. REG. PERKARA PDM-523/JEMBRA...
3,Minutasi,78/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan","Kamis, 01 Agu. 2024",35 Hari,[HEPY HARDIANTO PURBA],[Lailani Rahma Indah],Ida Bagus Made Ari Suamba,5,[HEPY HARDIANTO PURBA~Pidana Penjara Waktu Ter...,MENGADILI:\nMenyatakan Terdakwa Hepy Hardianto...,“Demi Keadilan Dan Kebenaran\nBerdasarkan Ketu...,"1 (satu) unit mobil Mitsubishi Xpander, warna ...",Demi Keadilan Dan Kebenaran\nBerdasarkan Ketuh...
5,Minutasi,76/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 18 Jul. 2024",40 Hari,"[RONY SETIAWAN, DIAN HERMAWAN]","[Miranda Widyawati,S.H., Kadek Cintyadewi Perm...",Ni Gusti Made Utami,4,[RONY SETIAWAN~Pidana Penjara Waktu Tertentu (...,MENGADILI:\nMenyatakan Terdakwa I Rony Setiawa...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik berisi kristal bening na...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...


##### Remove Header

In [66]:
pattern = r"(?i)dakwaan(?!\nnomor)[\s\S]*?(?=(?:dakwaan|$))"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.findall(pattern, text_dakwaan, re.DOTALL | re.IGNORECASE)

    if matches: 
        last_match = matches[-1]
        df_copy.loc[index, "cleaned_dakwaan"] = last_match.strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 109
raw dakwaan copied at 110
raw dakwaan copied at 111
raw dakwaan copied at 133
raw dakwaan copied at 134
raw dakwaan copied at 135
raw dakwaan copied at 136
raw dakwaan copied at 140
raw dakwaan copied at 141
raw dakwaan copied at 142
raw dakwaan copied at 143
raw dakwaan copied at 144
raw dakwaan copied at 145
raw dakwaan copied at 146
raw dakwaan copied at 147
raw dakwaan copied at 148
raw dakwaan copied at 149
raw dakwaan copied at 151
raw dakwaan copied at 152
raw dakwaan copied at 153
raw dakwaan copied at 154
raw dakwaan copied at 155
raw dakwaan copied at 156
raw dakwaan copied at 157
raw dakwaan copied at 158
raw dakwaan copied at 159
raw dakwaan copied at 160
raw dakwaan copied at 161
raw dakwaan copied at 162
raw dakwaan copied at 163
raw dakwaan copied at 164
raw dakwaan copied at 165
raw dakwaan copied at 166
raw dakwaan copied at 167
raw dakwaan copied at 168
raw dakwaan copied at 169
raw dakwaan copied at 170
raw dakwaan copied at 171
raw dakwaan 

In [67]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

DAKWAAN
PERTAMA
-Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut
 Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi Ilham Aris Darmawan akan be

##### Remove Footer

In [68]:
pattern = r"^(.*?)\s+Singaraja,\s+\d{1,2}\s+\w+\s+\d{4}"

for index, row in df_copy.iterrows():
    text_dakwaan = row['cleaned_dakwaan']
    matches = re.match(pattern, text_dakwaan, re.DOTALL)

    if matches: 
        df_copy.loc[index, "cleaned_dakwaan"] = matches.group(1).strip()
    else:
        df_copy.loc[index, "cleaned_dakwaan"] = text_dakwaan
        print(f"raw dakwaan copied at {index}")


raw dakwaan copied at 0
raw dakwaan copied at 1
raw dakwaan copied at 2
raw dakwaan copied at 3
raw dakwaan copied at 5
raw dakwaan copied at 6
raw dakwaan copied at 7
raw dakwaan copied at 8
raw dakwaan copied at 9
raw dakwaan copied at 10
raw dakwaan copied at 11
raw dakwaan copied at 12
raw dakwaan copied at 13
raw dakwaan copied at 14
raw dakwaan copied at 15
raw dakwaan copied at 16
raw dakwaan copied at 17
raw dakwaan copied at 18
raw dakwaan copied at 19
raw dakwaan copied at 21
raw dakwaan copied at 22
raw dakwaan copied at 23
raw dakwaan copied at 24
raw dakwaan copied at 25
raw dakwaan copied at 26
raw dakwaan copied at 27
raw dakwaan copied at 28
raw dakwaan copied at 29
raw dakwaan copied at 30
raw dakwaan copied at 31
raw dakwaan copied at 32
raw dakwaan copied at 33
raw dakwaan copied at 34
raw dakwaan copied at 35
raw dakwaan copied at 36
raw dakwaan copied at 37
raw dakwaan copied at 38
raw dakwaan copied at 40
raw dakwaan copied at 42
raw dakwaan copied at 43
raw dakwa

In [69]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

DAKWAAN
PERTAMA
-Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut
 Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi Ilham Aris Darmawan akan be

In [70]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace('\n', '. ', regex=False)

In [71]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut.  Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi Ilham Aris Darmawan akan

In [72]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace('  ', ' ', regex=False)

In [73]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut. Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi Ilham Aris Darmawan akan 

In [74]:
df_copy['cleaned_dakwaan'] = df_copy['cleaned_dakwaan'].str.replace(' .', '.', regex=False)

In [75]:
print(df_copy.iloc[1]['cleaned_dakwaan'])

DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH Alias ATUL pada hari Jumat tanggal 31 Mei 2024 sekira pukul 21.00 Wita atau setidak-tidaknya masih dalam bulan Mei 2024 bertempat di Jalan Danau Toba, Gang I, Lingkungan Terusan, Kelurahan Lelateng, Kabupaten Jembrana atau setidak-tidaknya pada suatu tempat lain yang masih termasuk dalam daerah hukum Pengadilan Negeri Negara yang berwenang mengadili melakukan tindak pidana, setiap orang yang tanpa hak atau melawan hukum memiliki, menyimpan, menguasai atau menyediakan Narkotika Golongan I bukan tanaman, yang dilakukan dengan cara sebagai berikut. Bahwa pada hari Kamis tanggal 30 Mei 2024 sekira pukul 19.00 wita terdakwa bertemu dengan saksi Ilham Aris Darmawan dan istrinya dirumah terdakwa di Perumahan Huma Lestari Bauk, Desa Baluk, Kecamatan Negara saat sedang mengobrol dengan istri siri terdakwa, kemudian saksi Ilham Aris Darmawan disuruh oleh istrinya untuk membeli nasi di daerah Loloan Barat dan pada saat saksi Ilham Aris Darmawan akan 

In [76]:
df_copy.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

In [77]:
print(f"Jumlah data setelah proses cleaning kolom Dakwaan: {df_copy.shape[0]}")

Jumlah data setelah proses cleaning kolom Dakwaan: 548


### Save to CSV - STAGE 1 PREPROCESSING

In [78]:
df_copy.to_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_Jembrana_FINAL.csv', index=False)

## Data Transformation

#### Data Transformation: Import Data

In [79]:
df_clean = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_Jembrana_FINAL.csv')
df_clean.head(3)

,status_perkara,nomor_perkara,klasifikasi_perkara,tanggal_pendaftaran,lama_proses,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,barang_bukti,dakwaan,cleaned_barang_bukti,cleaned_dakwaan
0,Minutasi,94/Pid.B/2024/PN Nga,Pencurian,"Rabu, 21 Agu. 2024",29 Hari,['SUDARMAN'],"['Lailani Rahma Indah', 'Kadek Cintyadewi Perm...",Ida Bagus Made Ari Suamba,5,['SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bu...,MENGADILI:\n1. Menyatakan Terdakwa Sudarman te...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,- 1 (satu) unit sepeda motor Honda Scopy tahun...,DAKWAAN. -Bahwa terdakwa SUDARMAN pada hari Sa...
1,Minutasi,82/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,['FATHULLAH als. ATUL'],"['Ni Wayan Iustikasari,S.H.', 'Selma Nabillah,...",Gde Putu Oka Yoga Bharata,3,['FATHULLAH als. ATUL~Pidana Penjara Waktu Ter...,MENGADILI:\nMenyatakan Terdakwa FATHULLAH Alia...,KEJAKSAAN REPUBLIK INDONESIA\nKEJAKSAAN TINGGI...,1 (satu) buah plastik klip berisi kristal beni...,DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH...
2,Minutasi,83/Pid.Sus/2024/PN Nga,Narkotika,"Kamis, 08 Agu. 2024",35 Hari,['I MADE SUDI ARTANA YASA'],"['Delfi Trimariono, S.H.', 'Kadek Cintyadewi P...",Ni Gusti Made Utami,3,['I MADE SUDI ARTANA YASA~Pidana Penjara Waktu...,MENGADILI:\nMenyatakan Terdakwa I Made Sudiart...,SURAT DAKWAAN\nNo. REG. PERKARA: PDM-523/JEMBR...,1. 14 (empat belas) buah plastik klip berisi k...,DAKWAAN. PERTAMA. -Bahwa Terdakwa I MADE SUDIA...


In [80]:
df_clean.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

In [81]:
null_rows = df_clean[df_clean.isnull().any(axis=1)]
print("Rows with null values:")
print(null_rows)

Rows with null values:
Empty DataFrame
Columns: [status_perkara, nomor_perkara, klasifikasi_perkara, tanggal_pendaftaran, lama_proses, terdakwa, penuntut_umum, hakim, jumlah_saksi, putusan_hukuman, barang_bukti, dakwaan, cleaned_barang_bukti, cleaned_dakwaan]
Index: []


#### Data Transformation: Drop Null Values

In [82]:
df_clean.dropna(inplace=True)

In [83]:
df_clean.isna().sum()

status_perkara          0
nomor_perkara           0
klasifikasi_perkara     0
tanggal_pendaftaran     0
lama_proses             0
terdakwa                0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
putusan_hukuman         0
barang_bukti            0
dakwaan                 0
cleaned_barang_bukti    0
cleaned_dakwaan         0
dtype: int64

#### Data Transformation: Drop Unused Columns

In [84]:
selected_columns = ['nomor_perkara', 
                    'klasifikasi_perkara', 
                    'terdakwa', 
                    'penuntut_umum', 
                    'hakim', 
                    'jumlah_saksi', 
                    'putusan_hukuman', 
                    'cleaned_barang_bukti', 
                    'cleaned_dakwaan']

df_clean = df_clean[selected_columns].copy()

df_clean.head(3)

,nomor_perkara,klasifikasi_perkara,terdakwa,penuntut_umum,hakim,jumlah_saksi,putusan_hukuman,cleaned_barang_bukti,cleaned_dakwaan
0,94/Pid.B/2024/PN Nga,Pencurian,['SUDARMAN'],"['Lailani Rahma Indah', 'Kadek Cintyadewi Perm...",Ida Bagus Made Ari Suamba,5,['SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bu...,- 1 (satu) unit sepeda motor Honda Scopy tahun...,DAKWAAN. -Bahwa terdakwa SUDARMAN pada hari Sa...
1,82/Pid.Sus/2024/PN Nga,Narkotika,['FATHULLAH als. ATUL'],"['Ni Wayan Iustikasari,S.H.', 'Selma Nabillah,...",Gde Putu Oka Yoga Bharata,3,['FATHULLAH als. ATUL~Pidana Penjara Waktu Ter...,1 (satu) buah plastik klip berisi kristal beni...,DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH...
2,83/Pid.Sus/2024/PN Nga,Narkotika,['I MADE SUDI ARTANA YASA'],"['Delfi Trimariono, S.H.', 'Kadek Cintyadewi P...",Ni Gusti Made Utami,3,['I MADE SUDI ARTANA YASA~Pidana Penjara Waktu...,1. 14 (empat belas) buah plastik klip berisi k...,DAKWAAN. PERTAMA. -Bahwa Terdakwa I MADE SUDIA...


### Data Transformation: Extract Multiple Terdakwa & Putusan Hukuman to each Row 

#### Check Data Type (Expected List)

In [85]:
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa string: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, str))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa string: {count_str_terdakwa}")

Jumlah Data: 548
Jumlah data dengan 'terdakwa' berupa string: 548
Jumlah data dengan 'putusan_hukuman' berupa string: 548


#### Convert To List

In [86]:
def clean_and_convert_to_list(terdakwa):
    if isinstance(terdakwa, list):
        return terdakwa
    
    if isinstance(terdakwa, str):
        cleaned_str = re.sub(r'\s+', ' ', terdakwa.strip())
        try:

            result_list = ast.literal_eval(cleaned_str)
        except (ValueError, SyntaxError):
            result_list = [cleaned_str]

        if isinstance(result_list, str):
            return [result_list]
        return result_list
    return []

df_clean['terdakwa'] = df_clean['terdakwa'].apply(clean_and_convert_to_list)
df_clean['putusan_hukuman'] = df_clean['putusan_hukuman'].apply(clean_and_convert_to_list)

In [87]:
print(df_clean['terdakwa'].apply(type).unique())
print(df_clean['putusan_hukuman'].apply(type).unique())
print(f"Jumlah Data: {len(df_clean)}")

count_str_terdakwa = df_clean[df_clean["terdakwa"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'terdakwa' berupa list: {count_str_terdakwa}")

count_str_putusan_hukuman = df_clean[df_clean["putusan_hukuman"].apply(lambda x: isinstance(x, list))].shape[0]
print(f"Jumlah data dengan 'putusan_hukuman' berupa list: {count_str_terdakwa}")

[<class 'list'>]
[<class 'list'>]
Jumlah Data: 548
Jumlah data dengan 'terdakwa' berupa list: 548
Jumlah data dengan 'putusan_hukuman' berupa list: 548


In [88]:
print(df_clean.loc[18, "putusan_hukuman"])
print(df_clean.loc[18, "terdakwa"])

['YULI ASTUTI alias YULI~Pidana Penjara Waktu Tertentu (1 Tahun )']
['YULI ASTUTI alias YULI']


#### Check if Jumlah terdakwa tidak sama dengan Jumlah Putusan (More Data Cleaning)

In [90]:
df_clean['terdakwa_len'] = df_clean['terdakwa'].apply(len)
df_clean['putusan_len'] = df_clean['putusan_hukuman'].apply(len)

mismatched_rows = df_clean[df_clean['terdakwa_len'] != df_clean['putusan_len']]
print(mismatched_rows[['terdakwa_len', 'putusan_len']].head())


Empty DataFrame
Columns: [terdakwa_len, putusan_len]
Index: []


In [91]:
print(mismatched_rows)

Empty DataFrame
Columns: [nomor_perkara, klasifikasi_perkara, terdakwa, penuntut_umum, hakim, jumlah_saksi, putusan_hukuman, cleaned_barang_bukti, cleaned_dakwaan, terdakwa_len, putusan_len]
Index: []


#### Copy Only if Terdakwa and Putusan is Same 

In [92]:
df_clean = df_clean[df_clean['terdakwa_len'] == df_clean['putusan_len']]
print(df_clean.shape)

(548, 11)


#### Explode Terdakwa and Putusan

In [93]:
def match_terdakwa_to_putusan(df):
    combined_terdakwa_putusan_list = []
    for index, row in df.iterrows():
        for terdakwa in row['terdakwa']:
            matched = False
            for putusan_hukuman in row['putusan_hukuman']:
                if terdakwa in putusan_hukuman:
                    temp_list = [row['nomor_perkara'], row['klasifikasi_perkara'], row['penuntut_umum'], row['hakim'], row['jumlah_saksi'], row['cleaned_barang_bukti'], row['cleaned_dakwaan'],terdakwa, putusan_hukuman] 
                    combined_terdakwa_putusan_list.append(temp_list)
                    matched = True
                    break
            if not matched:
                print(f'no match {terdakwa} at index {index}')
    df_combined = pd.DataFrame(combined_terdakwa_putusan_list, columns=['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'putusan_hukuman'])

    return df_combined

df_clean_exploded = match_terdakwa_to_putusan(df_clean)
print(df_clean_exploded)

               nomor_perkara                    klasifikasi_perkara  \
0       94/Pid.B/2024/PN Nga                              Pencurian   
1     82/Pid.Sus/2024/PN Nga                              Narkotika   
2     83/Pid.Sus/2024/PN Nga                              Narkotika   
3       78/Pid.B/2024/PN Nga  Penadahan, Penerbitan, dan Pencetakan   
4     76/Pid.Sus/2024/PN Nga                              Narkotika   
..                       ...                                    ...   
652  104/Pid.Sus/2017/PN Nga                            Lalu Lintas   
653   98/Pid.Sus/2017/PN Nga                              Narkotika   
654     99/Pid.B/2017/PN Nga                            Penggelapan   
655     96/Pid.B/2017/PN Nga                            Penggelapan   
656     94/Pid.B/2017/PN Nga                            Penggelapan   

                                         penuntut_umum  \
0    ['Lailani Rahma Indah', 'Kadek Cintyadewi Perm...   
1    ['Ni Wayan Iustikasari,S.H

In [94]:
df_clean_exploded[['nomor_perkara', 'terdakwa', 'putusan_hukuman']].head(5)

,nomor_perkara,terdakwa,putusan_hukuman
0,94/Pid.B/2024/PN Nga,SUDARMAN,SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bulan )
1,82/Pid.Sus/2024/PN Nga,FATHULLAH als. ATUL,FATHULLAH als. ATUL~Pidana Penjara Waktu Terte...
2,83/Pid.Sus/2024/PN Nga,I MADE SUDI ARTANA YASA,I MADE SUDI ARTANA YASA~Pidana Penjara Waktu T...
3,78/Pid.B/2024/PN Nga,HEPY HARDIANTO PURBA,HEPY HARDIANTO PURBA~Pidana Penjara Waktu Tert...
4,76/Pid.Sus/2024/PN Nga,RONY SETIAWAN,RONY SETIAWAN~Pidana Penjara Waktu Tertentu (4...


In [95]:
print(df_clean.shape)
print(df_clean_exploded.shape)
print(df_clean_exploded.isna().sum())

(548, 11)
(657, 9)
nomor_perkara           0
klasifikasi_perkara     0
penuntut_umum           0
hakim                   0
jumlah_saksi            0
cleaned_barang_bukti    0
cleaned_dakwaan         0
terdakwa                0
putusan_hukuman         0
dtype: int64


In [96]:
print(len(df_clean_exploded['terdakwa']))
print(len(df_clean_exploded['putusan_hukuman']))

657
657


In [97]:
for i in range(10):
    print(i, ".", df_clean_exploded.iloc[i]['putusan_hukuman'])

0 . SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bulan )
1 . FATHULLAH als. ATUL~Pidana Penjara Waktu Tertentu (4 Tahun )
Subsider Penjara (6 Bulan )
Subsider Denda Rp.800.000.000,00
2 . I MADE SUDI ARTANA YASA~Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )
Pidana Denda Rp.800.000.000,00
Subsider Penjara (2 Bulan )
3 . HEPY HARDIANTO PURBA~Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)
4 . RONY SETIAWAN~Pidana Penjara Waktu Tertentu (4 Tahun )
Pidana Denda Rp.800.000.000,00
Subsider Penjara (3 Bulan )
5 . DIAN HERMAWAN~Pidana Penjara Waktu Tertentu (4 Tahun )
Pidana Denda Rp.800.000.000,00
Subsider Penjara (3 Bulan )
6 . RA’IS MAULANA~Pidana Penjara Waktu Tertentu (4 Tahun )
Pidana Denda Rp.800.000.000,00
Subsider Penjara (1 Bulan )
7 . I KOMANG ARDIASA Als MANG MING~Pidana Penjara Waktu Tertentu (1 Tahun )
8 . I KOMANG ARDIASA Als MANG ELA~Pidana Penjara Waktu Tertentu (8 Bulan )
9 . BIMA ARFIANSYAH~Pidana Penjara Waktu Tertentu (5 Tahun 6 Bulan )
Subsider Kurungan (6 Bulan )
Subsider

### Data transformation: Extract Multiple Penuntut Umum

#### Check Data Type (Expected List)

In [98]:
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'str'>]


#### Convert to List

In [99]:
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [100]:
unique_types = df_clean_exploded['penuntut_umum'].apply(type).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [<class 'list'>]


#### Check Penuntut Umum > 1

In [101]:
unique_types = df_clean_exploded['penuntut_umum'].apply(len).unique()

print(f"Unique data types in the column: {unique_types}")

Unique data types in the column: [2 1 3]


In [102]:
list_lengths = df_clean_exploded['penuntut_umum'].apply(len)

unique_lengths = list_lengths.unique()

print(f"Unique lengths of data in the 'penuntut_umum' column: {unique_lengths}")
print("Indices for each unique length:")

for length in unique_lengths:
    indices = list_lengths[list_lengths == length].index
    print(f"Length {length}: Indices - {indices.tolist()}")


Unique lengths of data in the 'penuntut_umum' column: [2 1 3]
Indices for each unique length:
Length 2: Indices - [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 126, 127, 128, 129, 130, 131, 132, 133, 134, 136, 137, 138, 139, 140, 142, 143, 144, 145, 146, 147, 152, 153, 169]
Length 1: Indices - [3, 46, 100, 108, 123, 124, 125, 148, 149, 150, 151, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 19

In [103]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

0 ['Lailani Rahma Indah', 'Kadek Cintyadewi Permana,S.H.']
1 ['Ni Wayan Iustikasari,S.H.', 'Selma Nabillah,S.H.']
2 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
4 ['Miranda Widyawati,S.H.', 'Kadek Cintyadewi Permana,S.H.']
5 ['Miranda Widyawati,S.H.', 'Kadek Cintyadewi Permana,S.H.']
6 ['Lailani Rahma Indah', 'IDA Bagus Gede Eka Permana Putra,S.H.']
7 ['Kadek Cintyadewi Permana,S.H.', 'Selma Nabillah,S.H.']
8 ['Kadek Cintyadewi Permana,S.H.', 'Selma Nabillah,S.H.']
9 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
10 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
11 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
12 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
13 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
14 ['Ni Wayan Deasy Sriaryani, S.H.', 'IDA Bagus Gede Eka Permana Putra,S.H.']
15 ['Sofyan Heru,S.H.,M.H.', 'IDA Bagus Gede Eka Permana Putra,S.H.']
16 ['Miranda Widyawati,S.H.', 'Selma Nabillah,S.H.']

#### Fix Duplicates Penuntut Umum

In [104]:
# df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set([name.lower() for name in x])))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: list(set(x)))
df_clean_exploded['penuntut_umum'] = df_clean_exploded['penuntut_umum'].apply(lambda x: sorted(x))

In [105]:
for index, row in df_clean_exploded.iterrows():
    data = row['penuntut_umum']
    if len(data) > 1:
        print(index, row['penuntut_umum'])

0 ['Kadek Cintyadewi Permana,S.H.', 'Lailani Rahma Indah']
1 ['Ni Wayan Iustikasari,S.H.', 'Selma Nabillah,S.H.']
2 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
4 ['Kadek Cintyadewi Permana,S.H.', 'Miranda Widyawati,S.H.']
5 ['Kadek Cintyadewi Permana,S.H.', 'Miranda Widyawati,S.H.']
6 ['IDA Bagus Gede Eka Permana Putra,S.H.', 'Lailani Rahma Indah']
7 ['Kadek Cintyadewi Permana,S.H.', 'Selma Nabillah,S.H.']
8 ['Kadek Cintyadewi Permana,S.H.', 'Selma Nabillah,S.H.']
9 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
10 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
11 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
12 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
13 ['Delfi Trimariono, S.H.', 'Kadek Cintyadewi Permana,S.H.']
14 ['IDA Bagus Gede Eka Permana Putra,S.H.', 'Ni Wayan Deasy Sriaryani, S.H.']
15 ['IDA Bagus Gede Eka Permana Putra,S.H.', 'Sofyan Heru,S.H.,M.H.']
16 ['Miranda Widyawati,S.H.', 'Selma Nabillah,S.H.']

#### Explode DataFrame

In [106]:
df_clean_penuntut_exploded = df_clean_exploded.explode('penuntut_umum', ignore_index=True)

In [107]:
df_clean_penuntut_exploded[['nomor_perkara', 'terdakwa', 'penuntut_umum', 'putusan_hukuman']].head(5)

,nomor_perkara,terdakwa,penuntut_umum,putusan_hukuman
0,94/Pid.B/2024/PN Nga,SUDARMAN,"Kadek Cintyadewi Permana,S.H.",SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bulan )
1,94/Pid.B/2024/PN Nga,SUDARMAN,Lailani Rahma Indah,SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bulan )
2,82/Pid.Sus/2024/PN Nga,FATHULLAH als. ATUL,"Ni Wayan Iustikasari,S.H.",FATHULLAH als. ATUL~Pidana Penjara Waktu Terte...
3,82/Pid.Sus/2024/PN Nga,FATHULLAH als. ATUL,"Selma Nabillah,S.H.",FATHULLAH als. ATUL~Pidana Penjara Waktu Terte...
4,83/Pid.Sus/2024/PN Nga,I MADE SUDI ARTANA YASA,"Delfi Trimariono, S.H.",I MADE SUDI ARTANA YASA~Pidana Penjara Waktu T...


In [109]:
print(f"Jumlah data setelah proses transformasi penuntut umum: {df_clean_penuntut_exploded.shape[0]}")

Jumlah data setelah proses transformasi penuntut umum: 805


### Data Transformation: Extract Lama Kurungan Penjara

In [110]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                                                         SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bulan )
1                                                                                         SUDARMAN~Pidana Penjara Waktu Tertentu (5 Bulan )
2               FATHULLAH als. ATUL~Pidana Penjara Waktu Tertentu (4 Tahun )\nSubsider Penjara (6 Bulan )\nSubsider Denda Rp.800.000.000,00
3               FATHULLAH als. ATUL~Pidana Penjara Waktu Tertentu (4 Tahun )\nSubsider Penjara (6 Bulan )\nSubsider Denda Rp.800.000.000,00
4     I MADE SUDI ARTANA YASA~Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )\nPidana Denda Rp.800.000.000,00\nSubsider Penjara (2 Bulan )
5     I MADE SUDI ARTANA YASA~Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )\nPidana Denda Rp.800.000.000,00\nSubsider Penjara (2 Bulan )
6                                                                      HEPY HARDIANTO PURBA~Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)
7                   

#### Remove Terdakwa Name from Putusan Hukuman

In [111]:
df_clean_penuntut_exploded['cleaned_putusan_hukuman'] = df_clean_penuntut_exploded['putusan_hukuman'].apply(lambda x: x[0].split("~")[1] if isinstance(x, list) and len(x) > 0 else x.split("~")[1])

In [112]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'cleaned_putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                                          Pidana Penjara Waktu Tertentu (5 Bulan )
1                                                                          Pidana Penjara Waktu Tertentu (5 Bulan )
2           Pidana Penjara Waktu Tertentu (4 Tahun )\nSubsider Penjara (6 Bulan )\nSubsider Denda Rp.800.000.000,00
3           Pidana Penjara Waktu Tertentu (4 Tahun )\nSubsider Penjara (6 Bulan )\nSubsider Denda Rp.800.000.000,00
4     Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )\nPidana Denda Rp.800.000.000,00\nSubsider Penjara (2 Bulan )
5     Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )\nPidana Denda Rp.800.000.000,00\nSubsider Penjara (2 Bulan )
6                                                                   Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)
7             Pidana Penjara Waktu Tertentu (4 Tahun )\nPidana Denda Rp.800.000.000,00\nSubsider Penjara (3 Bulan )
8             Pidana Penjara Waktu Tertentu (4 Tahun )\nPidana Denda Rp.

#### Convert to List

In [113]:
df_clean_penuntut_exploded['cleaned_putusan_hukuman'] = df_clean_penuntut_exploded['cleaned_putusan_hukuman'].str.split('\n')

In [114]:
pd.set_option('display.max_colwidth', None)
print(df_clean_penuntut_exploded.loc[:10, 'cleaned_putusan_hukuman'])
pd.reset_option('display.max_colwidth')

0                                                                          [Pidana Penjara Waktu Tertentu (5 Bulan )]
1                                                                          [Pidana Penjara Waktu Tertentu (5 Bulan )]
2           [Pidana Penjara Waktu Tertentu (4 Tahun ), Subsider Penjara (6 Bulan ), Subsider Denda Rp.800.000.000,00]
3           [Pidana Penjara Waktu Tertentu (4 Tahun ), Subsider Penjara (6 Bulan ), Subsider Denda Rp.800.000.000,00]
4     [Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan ), Pidana Denda Rp.800.000.000,00, Subsider Penjara (2 Bulan )]
5     [Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan ), Pidana Denda Rp.800.000.000,00, Subsider Penjara (2 Bulan )]
6                                                                   [Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)]
7             [Pidana Penjara Waktu Tertentu (4 Tahun ), Pidana Denda Rp.800.000.000,00, Subsider Penjara (3 Bulan )]
8             [Pidana Penjara Waktu Tertentu (4 Tahun ),

#### Extract Pidana Penjara and Pidana Kurungan Text

In [115]:
pattern = r'Pidana Penjara'
pattern_2 = r'Pidana Kurungan'

for index, row in df_clean_penuntut_exploded.iterrows():
    list_putusan_hukuman = row['cleaned_putusan_hukuman']
    
    for putusan_hukuman in list_putusan_hukuman:
        match = re.search(pattern, putusan_hukuman, re.IGNORECASE)
        match_2 = re.search(pattern_2, putusan_hukuman, re.IGNORECASE)

        if match or match_2:
            df_clean_penuntut_exploded.loc[index, 'extracted_putusan_hukuman'] = putusan_hukuman
            # print(f"{index} -> {putusan_hukuman}")

#### Check Null Extracted Putusan Hukuman

In [109]:
null_values = df_clean_penuntut_exploded[df_clean_penuntut_exploded['extracted_putusan_hukuman'].isna()]
null_values[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']]

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
169,116/Pid.Sus/2023/PN Nga,Lain-Lain,"[Pidana Bersyarat, Pidana Denda Rp.1.000.000,0...",NaN
170,116/Pid.Sus/2023/PN Nga,Lain-Lain,"[Pidana Bersyarat, Pidana Denda Rp.1.000.000,0...",NaN
384,107/Pid.Sus/2021/PN Nga,Lalu Lintas,[Lain-lain],NaN
388,104/Pid.Sus/2021/PN Nga,Lalu Lintas,"[Pidana Denda Rp.10.000.000,00, Subsider Kurun...",NaN
448,37/Pid.B/2021/PN Nga,Penggelapan,[Subsider Penjara (1 Tahun 10 Bulan )],NaN
464,24/Pid.Sus/2021/PN Nga,Kesehatan,"[Pidana Denda Rp.15.000.000,00, Subsider Kurun...",NaN
578,152/Pid.Sus/2019/PN Nga,Lalu Lintas,"[Pidana Denda Rp.1.000.000,00, Subsider Kurung...",NaN
629,85/Pid.Sus/2019/PN Nga,Kesehatan,"[Subsider Kurungan (3 Bulan ), Subsider Denda ...",NaN
723,54/Pid.Sus/2018/PN Nga,Lain-Lain,[Pidana Bersyarat],NaN
735,26/Pid.Sus/2018/PN Nga,Lain-Lain,[Pidana Bersyarat],NaN


#### Use Subsider Penjara to Fill Null Values

In [116]:
pattern = r'Subsider'

for index, row in df_clean_penuntut_exploded.iterrows():
    list_putusan_hukuman = row['cleaned_putusan_hukuman']

    if row['extracted_putusan_hukuman'] is np.nan:
        for putusan_hukuman in list_putusan_hukuman:
            match = re.search(pattern, putusan_hukuman, re.IGNORECASE)

            if match:
                df_clean_penuntut_exploded.loc[index, 'extracted_putusan_hukuman'] = putusan_hukuman
                # print(f"{index} -> {putusan_hukuman}")
    

In [117]:
df_clean_penuntut_exploded[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']].head(10)

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
0,94/Pid.B/2024/PN Nga,Pencurian,[Pidana Penjara Waktu Tertentu (5 Bulan )],Pidana Penjara Waktu Tertentu (5 Bulan )
1,94/Pid.B/2024/PN Nga,Pencurian,[Pidana Penjara Waktu Tertentu (5 Bulan )],Pidana Penjara Waktu Tertentu (5 Bulan )
2,82/Pid.Sus/2024/PN Nga,Narkotika,"[Pidana Penjara Waktu Tertentu (4 Tahun ), Sub...",Pidana Penjara Waktu Tertentu (4 Tahun )
3,82/Pid.Sus/2024/PN Nga,Narkotika,"[Pidana Penjara Waktu Tertentu (4 Tahun ), Sub...",Pidana Penjara Waktu Tertentu (4 Tahun )
4,83/Pid.Sus/2024/PN Nga,Narkotika,[Pidana Penjara Waktu Tertentu (4 Tahun 6 Bula...,Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )
5,83/Pid.Sus/2024/PN Nga,Narkotika,[Pidana Penjara Waktu Tertentu (4 Tahun 6 Bula...,Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )
6,78/Pid.B/2024/PN Nga,"Penadahan, Penerbitan, dan Pencetakan",[Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)],Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)
7,76/Pid.Sus/2024/PN Nga,Narkotika,"[Pidana Penjara Waktu Tertentu (4 Tahun ), Pid...",Pidana Penjara Waktu Tertentu (4 Tahun )
8,76/Pid.Sus/2024/PN Nga,Narkotika,"[Pidana Penjara Waktu Tertentu (4 Tahun ), Pid...",Pidana Penjara Waktu Tertentu (4 Tahun )
9,76/Pid.Sus/2024/PN Nga,Narkotika,"[Pidana Penjara Waktu Tertentu (4 Tahun ), Pid...",Pidana Penjara Waktu Tertentu (4 Tahun )


In [118]:
null_values = df_clean_penuntut_exploded[df_clean_penuntut_exploded['extracted_putusan_hukuman'].isna()]
null_values[['nomor_perkara', 'klasifikasi_perkara','cleaned_putusan_hukuman', 'extracted_putusan_hukuman']]

,nomor_perkara,klasifikasi_perkara,cleaned_putusan_hukuman,extracted_putusan_hukuman
384,107/Pid.Sus/2021/PN Nga,Lalu Lintas,[Lain-lain],NaN
723,54/Pid.Sus/2018/PN Nga,Lain-Lain,[Pidana Bersyarat],NaN
735,26/Pid.Sus/2018/PN Nga,Lain-Lain,[Pidana Bersyarat],NaN
736,23/Pid.B/2018/PN Nga,Kejahatan Perjudian,[Pidana Bersyarat],NaN
737,23/Pid.B/2018/PN Nga,Kejahatan Perjudian,[Pidana Bersyarat],NaN
738,23/Pid.B/2018/PN Nga,Kejahatan Perjudian,[Pidana Bersyarat],NaN
739,23/Pid.B/2018/PN Nga,Kejahatan Perjudian,[Pidana Bersyarat],NaN
755,135/Pid.B/2017/PN Nga,Kejahatan Perjudian,[Lain-lain],NaN
756,135/Pid.B/2017/PN Nga,Kejahatan Perjudian,[Lain-lain],NaN
757,135/Pid.B/2017/PN Nga,Kejahatan Perjudian,[Lain-lain],NaN


#### Drop Null Values

In [119]:
df_clean_penuntut_exploded.dropna(subset=['extracted_putusan_hukuman'], inplace=True)

In [120]:
df_clean_penuntut_exploded.isna().sum()

nomor_perkara                0
klasifikasi_perkara          0
penuntut_umum                0
hakim                        0
jumlah_saksi                 0
cleaned_barang_bukti         0
cleaned_dakwaan              0
terdakwa                     0
putusan_hukuman              0
cleaned_putusan_hukuman      0
extracted_putusan_hukuman    0
dtype: int64

#### Extract Lama Kurungan Penjara

In [123]:
df_extracted = df_clean_penuntut_exploded.copy()

pattern_tbh = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Bulan\s*(\d+)\s*Hari\s*\)?"
pattern_tb = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Bulan\s*\)?"
pattern_th = r"\(?\s*(\d+)\s*Tahun\s*(\d+)\s*Hari\s*\)?"
pattern_bh = r"\(?\s*(\d+)\s*Bulan\s*(\d+)\s*Hari\s*\)?"
pattern_tahun = r"\s*(\d+)\s*Tahun"
pattern_bulan = r"\s*(\d+)\s*Bulan"
pattern_hari = r"\s*(\d+)\s*Hari"

df_extracted['Tahun'] = None
df_extracted['Bulan'] = None
df_extracted['Hari'] = None

for index, row in df_extracted.iterrows():
    text = row["extracted_putusan_hukuman"]

    match_tbh = re.search(pattern_tbh, text, re.IGNORECASE)
    match_tb = re.search(pattern_tb, text, re.IGNORECASE)
    match_th = re.search(pattern_th, text, re.IGNORECASE)
    match_bh = re.search(pattern_bh, text, re.IGNORECASE)
    match_tahun = re.search(pattern_tahun, text, re.IGNORECASE)
    match_bulan = re.search(pattern_bulan, text, re.IGNORECASE)
    match_hari = re.search(pattern_hari, text, re.IGNORECASE)

    if match_tbh:
        tahun = int(match_tbh.group(1))
        bulan = int(match_tbh.group(2))
        hari = int(match_tbh.group(3))
        print(f"MATCH TBH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_tb:
        tahun = int(match_tb.group(1))
        bulan = int(match_tb.group(2))
        hari = 0
        print(f"MATCH TB PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_th:
        tahun = int(match_th.group(1))
        bulan = 0
        hari = int(match_th.group(2))
        print(f"MATCH TH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_bh:
        tahun = 0
        bulan = int(match_bh.group(1))
        hari = int(match_bh.group(2))
        print(f"MATCH BH PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_tahun:
        tahun = int(match_tahun.group(1))
        bulan = 0
        hari = 0
        print(f"MATCH TAHUN PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_bulan:
        tahun = 0
        bulan = int(match_bulan.group(1))
        hari = 0
        print(f"MATCH BULAN PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    elif match_hari:
        tahun = 0
        bulan = 0
        hari = int(match_hari.group(1))
        print(f"MATCH HARI PATTERN | {text} -> {tahun} Tahun, {bulan} Bulan, {hari} Hari")

    else:
        tahun = None
        bulan = None
        hari = None

    df_extracted.loc[index, 'Tahun'] = tahun
    df_extracted.loc[index, 'Bulan'] = bulan
    df_extracted.loc[index, 'Hari'] = hari

MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (5 Bulan ) -> 0 Tahun, 5 Bulan, 0 Hari
MATCH BULAN PATTERN | Pidana Penjara Waktu Tertentu (5 Bulan ) -> 0 Tahun, 5 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun ) -> 4 Tahun, 0 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun ) -> 4 Tahun, 0 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan ) -> 4 Tahun, 6 Bulan, 0 Hari
MATCH TB PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan ) -> 4 Tahun, 6 Bulan, 0 Hari
MATCH BH PATTERN | Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari) -> 0 Tahun, 3 Bulan, 20 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun ) -> 4 Tahun, 0 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun ) -> 4 Tahun, 0 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun ) -> 4 Tahun, 0 Bulan, 0 Hari
MATCH TAHUN PATTERN | Pidana Penjara Waktu Tertentu (4 Tahun ) -> 4 Tahun, 

In [124]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

print(df_extracted[["extracted_putusan_hukuman", "Tahun", "Bulan", "Hari"]])

pd.reset_option('display.max_rows')
pd.reset_option('display.max_colwidth')

                             extracted_putusan_hukuman Tahun Bulan  Hari
0             Pidana Penjara Waktu Tertentu (5 Bulan )     0     5     0
1             Pidana Penjara Waktu Tertentu (5 Bulan )     0     5     0
2             Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
3             Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
4     Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )     4     6     0
5     Pidana Penjara Waktu Tertentu (4 Tahun 6 Bulan )     4     6     0
6      Pidana Penjara Waktu Tertentu (3 Bulan 20 Hari)     0     3    20
7             Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
8             Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
9             Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
10            Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
11            Pidana Penjara Waktu Tertentu (4 Tahun )     4     0     0
12            Pidana Penjara Waktu Tertentu (4 Tahu

#### Check Null Values after Extracting Lama Penjara

In [125]:
df_extracted.isnull().any()

nomor_perkara                False
klasifikasi_perkara          False
penuntut_umum                False
hakim                        False
jumlah_saksi                 False
cleaned_barang_bukti         False
cleaned_dakwaan              False
terdakwa                     False
putusan_hukuman              False
cleaned_putusan_hukuman      False
extracted_putusan_hukuman    False
Tahun                         True
Bulan                         True
Hari                          True
dtype: bool

In [126]:
df_extracted.dropna(subset=['Tahun', 'Bulan', 'Hari'], inplace=True)

In [127]:
df_extracted.isnull().any()

nomor_perkara                False
klasifikasi_perkara          False
penuntut_umum                False
hakim                        False
jumlah_saksi                 False
cleaned_barang_bukti         False
cleaned_dakwaan              False
terdakwa                     False
putusan_hukuman              False
cleaned_putusan_hukuman      False
extracted_putusan_hukuman    False
Tahun                        False
Bulan                        False
Hari                         False
dtype: bool

#### Convert Kurungan Penjara menjadi Bulan

In [128]:
df_extracted['total_pidana_penjara_bulan'] = (df_extracted['Tahun'] * 12) + df_extracted['Bulan'] + (df_extracted['Hari'] / 30)

In [129]:
pd.set_option('display.max_rows', None)
print(df_extracted[["Tahun", "Bulan", "Hari", "total_pidana_penjara_bulan"]])
pd.reset_option('display.max_rows')

    Tahun Bulan Hari total_pidana_penjara_bulan
0       0     5    0                        5.0
1       0     5    0                        5.0
2       4     0    0                       48.0
3       4     0    0                       48.0
4       4     6    0                       54.0
5       4     6    0                       54.0
6       0     3   20                   3.666667
7       4     0    0                       48.0
8       4     0    0                       48.0
9       4     0    0                       48.0
10      4     0    0                       48.0
11      4     0    0                       48.0
12      4     0    0                       48.0
13      1     0    0                       12.0
14      1     0    0                       12.0
15      0     8    0                        8.0
16      0     8    0                        8.0
17      5     6    0                       66.0
18      5     6    0                       66.0
19      5     6    0                    

In [130]:
df_extracted.to_csv(f'../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_Jembrana_FINAL.csv', index=False)

In [131]:
print(f"Jumlah data setelah proses ekstraksi label: {df_extracted.shape[0]}")

Jumlah data setelah proses ekstraksi label: 787


## Data Validation

In [132]:
df_stage_1 = pd.read_csv(f'../Data/STAGE 1 PREPROCESSING/STAGE_1_PREPROCESSING_Jembrana_FINAL.csv')
df_stage_2 = pd.read_csv(f'../Data/STAGE 2 PREPROCESSING/STAGE_2_PREPROCESSING_Jembrana_FINAL.csv')

In [133]:
print(df_stage_1.columns)
print(df_stage_2.columns)

Index(['status_perkara', 'nomor_perkara', 'klasifikasi_perkara',
       'tanggal_pendaftaran', 'lama_proses', 'terdakwa', 'penuntut_umum',
       'hakim', 'jumlah_saksi', 'putusan_hukuman', 'barang_bukti', 'dakwaan',
       'cleaned_barang_bukti', 'cleaned_dakwaan'],
      dtype='object')
Index(['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim',
       'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa',
       'putusan_hukuman', 'cleaned_putusan_hukuman',
       'extracted_putusan_hukuman', 'Tahun', 'Bulan', 'Hari',
       'total_pidana_penjara_bulan'],
      dtype='object')


In [134]:
df_validation = df_stage_2[['nomor_perkara', 'klasifikasi_perkara', 'penuntut_umum', 'hakim', 'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan', 'terdakwa', 'total_pidana_penjara_bulan']]

In [135]:
df_validation.head(5)

,nomor_perkara,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,94/Pid.B/2024/PN Nga,Pencurian,"Kadek Cintyadewi Permana,S.H.",Ida Bagus Made Ari Suamba,5,- 1 (satu) unit sepeda motor Honda Scopy tahun...,DAKWAAN. -Bahwa terdakwa SUDARMAN pada hari Sa...,SUDARMAN,5.0
1,94/Pid.B/2024/PN Nga,Pencurian,Lailani Rahma Indah,Ida Bagus Made Ari Suamba,5,- 1 (satu) unit sepeda motor Honda Scopy tahun...,DAKWAAN. -Bahwa terdakwa SUDARMAN pada hari Sa...,SUDARMAN,5.0
2,82/Pid.Sus/2024/PN Nga,Narkotika,"Ni Wayan Iustikasari,S.H.",Gde Putu Oka Yoga Bharata,3,1 (satu) buah plastik klip berisi kristal beni...,DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH...,FATHULLAH als. ATUL,48.0
3,82/Pid.Sus/2024/PN Nga,Narkotika,"Selma Nabillah,S.H.",Gde Putu Oka Yoga Bharata,3,1 (satu) buah plastik klip berisi kristal beni...,DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH...,FATHULLAH als. ATUL,48.0
4,83/Pid.Sus/2024/PN Nga,Narkotika,"Delfi Trimariono, S.H.",Ni Gusti Made Utami,3,1. 14 (empat belas) buah plastik klip berisi k...,DAKWAAN. PERTAMA. -Bahwa Terdakwa I MADE SUDIA...,I MADE SUDI ARTANA YASA,54.0


#### Validate Data Types

In [136]:
for column in df_validation.columns:
    types_in_column = df_stage_2[column].apply(type).unique()
    print(f"Data types in column '{column}' -> {types_in_column}")

Data types in column 'nomor_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara' -> [<class 'str'>]
Data types in column 'penuntut_umum' -> [<class 'str'>]
Data types in column 'hakim' -> [<class 'str'>]
Data types in column 'jumlah_saksi' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan' -> [<class 'str'>]
Data types in column 'terdakwa' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'float'>]


#### Merge DataFrame

In [137]:
merged_df = df_validation.merge(df_stage_1, on='nomor_perkara', suffixes=('_stage_2', '_stage_1'))

In [138]:
for column in merged_df.columns:
    types_in_column = merged_df[column].apply(type).unique()
    print(f"Data types in column '{column}' -> {types_in_column}")

Data types in column 'nomor_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara_stage_2' -> [<class 'str'>]
Data types in column 'penuntut_umum_stage_2' -> [<class 'str'>]
Data types in column 'hakim_stage_2' -> [<class 'str'>]
Data types in column 'jumlah_saksi_stage_2' -> [<class 'int'>]
Data types in column 'cleaned_barang_bukti_stage_2' -> [<class 'str'>]
Data types in column 'cleaned_dakwaan_stage_2' -> [<class 'str'>]
Data types in column 'terdakwa_stage_2' -> [<class 'str'>]
Data types in column 'total_pidana_penjara_bulan' -> [<class 'float'>]
Data types in column 'status_perkara' -> [<class 'str'>]
Data types in column 'klasifikasi_perkara_stage_1' -> [<class 'str'>]
Data types in column 'tanggal_pendaftaran' -> [<class 'str'>]
Data types in column 'lama_proses' -> [<class 'str'>]
Data types in column 'terdakwa_stage_1' -> [<class 'str'>]
Data types in column 'penuntut_umum_stage_1' -> [<class 'str'>]
Data types in column 'hakim_stage_1' -> [<class 'str'>]
Da

In [139]:
merged_df.isna().sum()

nomor_perkara                   0
klasifikasi_perkara_stage_2     0
penuntut_umum_stage_2           0
hakim_stage_2                   0
jumlah_saksi_stage_2            0
cleaned_barang_bukti_stage_2    0
cleaned_dakwaan_stage_2         0
terdakwa_stage_2                0
total_pidana_penjara_bulan      0
status_perkara                  0
klasifikasi_perkara_stage_1     0
tanggal_pendaftaran             0
lama_proses                     0
terdakwa_stage_1                0
penuntut_umum_stage_1           0
hakim_stage_1                   0
jumlah_saksi_stage_1            0
putusan_hukuman                 0
barang_bukti                    0
dakwaan                         0
cleaned_barang_bukti_stage_1    0
cleaned_dakwaan_stage_1         0
dtype: int64

#### Validate Each Row Based on Nomor Perkara (Klasifikasi Perkara, Terdakwa, Penuntut Umum, Hakim, Jumlah Saksi)

##### Klasifikasi Perkara

In [140]:
merged_df['match_klasifikasi_perkara'] = merged_df['klasifikasi_perkara_stage_2'] == merged_df['klasifikasi_perkara_stage_1']

for _, row in merged_df.iterrows():
    if row['match_klasifikasi_perkara']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: KLASIFIKASI PERKARA MISMATCH!")

##### Terdakwa

In [141]:
for index, row in merged_df.iterrows():
    terdakwa_stage_2 = row['terdakwa_stage_2']

    if terdakwa_stage_2 in row['terdakwa_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: TERDAKWA MISMATCH!")

##### Penuntut Umum

In [142]:
for index, row in merged_df.iterrows():
    penuntut_umum_stage_2 = row['penuntut_umum_stage_2']

    if penuntut_umum_stage_2 in row['penuntut_umum_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: PENUNTUT UMUM MISMATCH!")


##### Hakim

In [143]:
for index, row in merged_df.iterrows():
    hakim_stage_2 = row['hakim_stage_2']

    if hakim_stage_2 in row['hakim_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: HAKIM MISMATCH!")

##### Jumlah Saksi

In [144]:
for index, row in merged_df.iterrows():
    if row['jumlah_saksi_stage_2'] == row['jumlah_saksi_stage_1']:
        continue
    else:
        print(f"NOMOR PERKARA {row['nomor_perkara']}: JUMLAH SAKSI MISMATCH!")

#### Drop Unnecessary Columns

In [145]:
merged_df.columns

Index(['nomor_perkara', 'klasifikasi_perkara_stage_2', 'penuntut_umum_stage_2',
       'hakim_stage_2', 'jumlah_saksi_stage_2', 'cleaned_barang_bukti_stage_2',
       'cleaned_dakwaan_stage_2', 'terdakwa_stage_2',
       'total_pidana_penjara_bulan', 'status_perkara',
       'klasifikasi_perkara_stage_1', 'tanggal_pendaftaran', 'lama_proses',
       'terdakwa_stage_1', 'penuntut_umum_stage_1', 'hakim_stage_1',
       'jumlah_saksi_stage_1', 'putusan_hukuman', 'barang_bukti', 'dakwaan',
       'cleaned_barang_bukti_stage_1', 'cleaned_dakwaan_stage_1',
       'match_klasifikasi_perkara'],
      dtype='object')

In [146]:
df_stage_3 = df_validation[['klasifikasi_perkara', 
                            'penuntut_umum', 
                            'hakim', 
                            'jumlah_saksi', 
                            'cleaned_barang_bukti', 
                            'cleaned_dakwaan', 
                            'terdakwa', 
                            'total_pidana_penjara_bulan']]

df_stage_3.to_csv(f'../Data/STAGE 3 PREPROCESSING/STAGE_3_PREPROCESSING_Jembrana_FINAL.csv', index=False)

In [150]:
print(f"Jumlah data setelah proses Data Validation: {df_extracted.shape[0]}")

Jumlah data setelah proses Data Validation: 787


#### Check Saved CSV

In [147]:
df_stage_3 = pd.read_csv(f'../Data/STAGE 3 PREPROCESSING/STAGE_3_PREPROCESSING_Jembrana_FINAL.csv')

In [148]:
df_stage_3.head(5)

,klasifikasi_perkara,penuntut_umum,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,terdakwa,total_pidana_penjara_bulan
0,Pencurian,"Kadek Cintyadewi Permana,S.H.",Ida Bagus Made Ari Suamba,5,- 1 (satu) unit sepeda motor Honda Scopy tahun...,DAKWAAN. -Bahwa terdakwa SUDARMAN pada hari Sa...,SUDARMAN,5.0
1,Pencurian,Lailani Rahma Indah,Ida Bagus Made Ari Suamba,5,- 1 (satu) unit sepeda motor Honda Scopy tahun...,DAKWAAN. -Bahwa terdakwa SUDARMAN pada hari Sa...,SUDARMAN,5.0
2,Narkotika,"Ni Wayan Iustikasari,S.H.",Gde Putu Oka Yoga Bharata,3,1 (satu) buah plastik klip berisi kristal beni...,DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH...,FATHULLAH als. ATUL,48.0
3,Narkotika,"Selma Nabillah,S.H.",Gde Putu Oka Yoga Bharata,3,1 (satu) buah plastik klip berisi kristal beni...,DAKWAAN. PERTAMA. -Bahwa ia terdakwa FATHULLAH...,FATHULLAH als. ATUL,48.0
4,Narkotika,"Delfi Trimariono, S.H.",Ni Gusti Made Utami,3,1. 14 (empat belas) buah plastik klip berisi k...,DAKWAAN. PERTAMA. -Bahwa Terdakwa I MADE SUDIA...,I MADE SUDI ARTANA YASA,54.0


In [149]:
df_stage_3.isna().sum()

klasifikasi_perkara           0
penuntut_umum                 0
hakim                         0
jumlah_saksi                  0
cleaned_barang_bukti          0
cleaned_dakwaan               0
terdakwa                      0
total_pidana_penjara_bulan    0
dtype: int64

# Text Summarization

In [172]:
print("CUDA available:", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("CUDA version:", torch.version.cuda)

CUDA available: True
Device count: 1
CUDA version: 12.1


In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    print(f"Number of CUDA devices: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("No CUDA devices available.")

In [4]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.1214, 0.0298, 0.0406],
        [0.8481, 0.8833, 0.1258],
        [0.8622, 0.3680, 0.7690],
        [0.6251, 0.0697, 0.8667],
        [0.2934, 0.7982, 0.9171]])


In [15]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Sep_12_02:55:00_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.77
Build cuda_12.6.r12.6/compiler.34841621_0


In [16]:
!nvidia-smi

Fri Dec  6 12:56:58 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 566.14                 Driver Version: 566.14         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   49C    P8              4W /   76W |     670MiB /   8188MiB |     13%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Text Summarization "t5-base-indonesian-summarization-cased"

In [5]:
from transformers import pipeline

from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
from nltk.translate.bleu_score import sentence_bleu
import torch
from bert_score import score
from nltk.tokenize import word_tokenize
from nltk.translate import meteor_score
import nltk

import time
from torch import float16

nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('punkt')


[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### CLEAN \N

In [6]:
df_extracted['cleaned_dakwaan'] = df_extracted['cleaned_dakwaan'].str.replace('\n', ' ', regex=True)

In [9]:
df_extracted['cleaned_dakwaan']

0       DAKWAAN   PERTAMA  - Bahwa Terdakwa KETUT SUBA...
1       DAKWAAN   PERTAMA  - Bahwa Terdakwa KETUT SUBA...
2       DAKWAAN   KESATU             Bahwa ia Terdakwa...
3       DAKWAAN   KESATU                              ...
4       Dakwaan Ke Satu - Bahwa  terdakwa  SANG PUTU W...
                              ...                        
1191    Kesatu  Pasal 83 ayat (1) huruf b UU RI No. 18...
1192    Perbuatan terdakwa sebagaimana diatur dan dian...
1193    Pasal 351 ayat (1) Kitab Undang-undang Hukum p...
1194    Pertama  Pasal 187 ayat (1) KUHP dan Kedua  Pa...
1195          Pasal 362 Kitab Undang-undang Hukum Pidana.
Name: cleaned_dakwaan, Length: 1196, dtype: object

In [ ]:
model_names = [
    "cahya/bert2bert-indonesian-summarization",
    "rowjak/bert-indonesian-news-summarization"
    "cahya/bert2gpt-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
]

In [ ]:


pipelines = {
    model_name: pipeline(
        "summarization",
        model=model_name,
        use_fast=True,
        device="cuda",
        batch_size=batch_size,
        torch_dtype="auto"
    )
    for model_name in model_names
}

In [ ]:
import warnings
from transformers import pipeline, AutoTokenizer
pipelines ={}

for model_name in model_names:
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,
    )

Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch: Sentencepiece tokenizer fallback detected. Using slow tokenizer.
Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3: Sentencepiece tokenizer fallback detected. Using slow tokenizer.


The encoder model config class: <class 'transformers.models.bert.configuration_bert.BertConfig'> is different from the decoder model config class: <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>. It is not recommended to use the `AutoTokenizer.from_pretrained()` method in this case. Please use the encoder and decoder specific tokenizer classes.
Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0

Warning for interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Sentencepiece tokenizer fallback detected. Using slow tokenizer.


Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

In [20]:
pipelines

{'cahya/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243ddf0ece0>,
 'panggi/t5-small-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e05602e0>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243d667f610>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e056dae0>,
 'cahya/bert2gpt-indonesian-summarization': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e54cbfd0>,
 'panggi/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243e54cbe20>,
 'rayendito/mt5-small-finetuned-xl-sum-indonesia': <transformers.pipelines.text2text_generation.SummarizationPipeline at 0x243d38195d0>,


In [17]:
import pprint

pprint.pprint(pipelines)

{'cahya/bert2gpt-indonesian-summarization': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243E04B5EA0>,
 'cahya/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D3818FA0>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4E7FA30>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D6723010>,
 'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4E3DCC0>,
 'panggi/t5-base-indonesian-summarization-cased': <transformers.pipelines.text2text_generation.SummarizationPipeline object at 0x00000243D4BB4880>,
 'panggi/t5-small-indonesian-summ

In [8]:
df_extracted

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN KESATU Bahwa ia Terdakwa...,52
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN KESATU ...,72
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48
...,...,...,...,...,...,...,...
1191,Perlindungan dan Pengelolaan Lingkungan Hidup,Jumat Ariyanto,Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12
1192,Kejahatan Perjudian,I KETUT ANA SAPUTRA Alias BENCUT,Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,4
1193,Penganiayaan,Sahabudin Alias Udin,Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,9
1194,Kejahatan yang Membahayakan Keamananan Umum Ba...,Putu Kristian Damanta,A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP dan Kedua Pa...,24


In [10]:
avg_length = df_extracted['cleaned_dakwaan'].apply(lambda text: len(text.split())).mean()

print(f"Average length of cleaned_dakwaan: {avg_length:.2f} words")

Average length of cleaned_dakwaan: 691.86 words


In [9]:
import datetime
import psutil
import os
def log_memory_usage():
    print(f"CPU usage: {psutil.cpu_percent()}%, Memory usage: {psutil.virtual_memory().percent}%")
    if torch.cuda.is_available():
        print(f"GPU memory allocated: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")
        print(f"GPU memory reserved: {torch.cuda.memory_reserved() / 1024 ** 2:.2f} MB")

save_dir = '../Data/STAGE 3 PREPROCESSING'

if not os.path.exists(save_dir):
    os.makedirs(save_dir)



In [10]:
import warnings
from transformers import pipeline, AutoTokenizer
import time
import datetime
import torch
from torch.amp import autocast

warnings.filterwarnings("ignore", category=UserWarning, module="transformers")
warnings.filterwarnings("ignore", message="Your max_length is set to \d+, but your input_length is only \d+", category=UserWarning, module="transformers")
chunk_size = 450
batch_size = 12

for model_name in model_names:
    pipelines = {}
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Capture all warnings
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True).to('cuda')

            # Check for the sentencepiece warning
            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    # Create the summarization pipeline
    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,  # Change this to `-1` if not using GPU
    )

    for key, pipe in pipelines.items():
        start_time = time.time()
        summarized_texts = []  # To store summaries for each row

        for index, row in df_extracted.iterrows():
            # print(f"Processing row {index + 1}/{len(df_extracted)} for model {key}")
            if index % 100 == 0:
                print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)} for model {key}")

            text = row['cleaned_dakwaan']

            # Split the text into manageable chunks
            chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            # Summarize each batch of chunks
            summaries = []
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]

                # Dynamically calculate max_length and min_length
                valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
                if valid_lengths:
                    batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
                    batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
                else:
                    batch_max_length = 100  # Default max for very short chunks
                    batch_min_length = 50  # Default min for very short chunks

                try:
                    with autocast(device_type='cuda'):  # Enable mixed-precision for efficiency
                        results = pipe(
                            batch,
                            max_length=batch_max_length,
                            min_length=batch_min_length,
                            do_sample=False,
                        )
                        summaries.extend([result.get("generated_text", result.get("summary_text", "")) for result in results])
                except Exception as e:
                    print(f"Error with model {key} on batch {i // batch_size}: {e}")
                    summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

            # Combine all summaries for the current row
            summarized_texts.append(" ".join(summaries))

        # Save summarized texts in the DataFrame
        df_extracted[f'{key}_summarization_dakwaan'] = summarized_texts

        # Save intermediate results to avoid data loss
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
        df_extracted.to_csv(save_dir, index=False)
        print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")

        # Log memory usage (assuming log_memory_usage is implemented elsewhere)
        # log_memory_usage()

        # Free GPU memory
        del pipe
        torch.cuda.empty_cache()
        print(f"Model {key} - Execution time: {time.time() - start_time:.2f} seconds\n")

    # Free pipelines dictionary
    del pipelines
    torch.cuda.empty_cache()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Fast tokenizer not supported for panggi/t5-base-indonesian-summarization-cased due to error: 'T5TokenizerFast' object has no attribute 'to'. Using slow tokenizer.
Processing row 1/1196 - 100/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your

Processing row 101/1196 - 200/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 85. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your max_length is set to 100, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 93. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=46)
Your

...

Processing row 1001/1196 - 1100/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 99. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=49)
Your max_length is set to 100, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)
Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 84. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=42)
Your m

Processing row 1101/1196 - 1200/1196 for model panggi/t5-base-indonesian-summarization-cased


Your max_length is set to 100, but your input_length is only 94. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=47)
Your max_length is set to 100, but your input_length is only 79. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)
Your max_length is set to 100, but your input_length is only 90. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=45)
Your max_length is set to 100, but your input_length is only 89. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)
Your


CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241208_055637.csv
Model panggi/t5-base-indonesian-summarization-cased - Execution time: 20091.55 seconds



In [100]:
df_tempor = pd.read_csv('../Data/STAGE 3 PREPROCESSING/Final Singaraja Summarization.csv')

In [108]:
df_tempor.columns

Index(['Unnamed: 0', 'klasifikasi_perkara', 'terdakwa', 'hakim',
       'jumlah_saksi', 'cleaned_barang_bukti', 'cleaned_dakwaan',
       'total_pidana_penjara_bulan',
       'panggi/t5-small-indonesian-summarization-cased_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3_summarization_dakwaan',
       'rayendito/mt5-small-finetuned-xl-sum-indonesia_summarization_dakwaan',
       'interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs_summarization_dakwaan',
       'panggi/t5-base-indonesian-summarization-cased_summarization_dakwaan',
       'cahya_bert2bert_summarization_dakwaan',
       'rowjak/bert-indonesian-news-summarization',
       'cahya_bert2gpt_summarization_dakwaan',
       'cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan'],
      dtype='object')

In [107]:
df_tempor.iloc[:, -12:]

,cleaned_dakwaan,total_pidana_penjara_bulan,panggi/t5-small-indonesian-summarization-cased_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3_summarization_dakwaan,rayendito/mt5-small-finetuned-xl-sum-indonesia_summarization_dakwaan,interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs_summarization_dakwaan,panggi/t5-base-indonesian-summarization-cased_summarization_dakwaan,cahya_bert2bert_summarization_dakwaan,rowjak/bert-indonesian-news-summarization,cahya_bert2gpt_summarization_dakwaan,cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan
0,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,Bahwa Terdakwa KETUT SUBAGIA bersama dengan te...,KeTUT SUBAGIA bersama dengan Terdakwa KOMANG A...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Seorang terdakwa kecelakaan pertama - telah me...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Pemain permainan judi bola adil b...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - bahwa terdakwa ketut subagia...,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
1,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,Bahwa Terdakwa KETUT SUBAGIA bersama dengan te...,KeTUT SUBAGIA bersama dengan Terdakwa KOMANG A...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Seorang terdakwa kecelakaan pertama - telah me...,Bahwa Terdakwa KETUT SUBAGIA bersama dengan Te...,Pemain permainan judi bola adil b...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - kedua terdakwa ketut subagia...,dakwaan pertama - bahwa terdakwa ketut subuku ...,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
2,DAKWAAN KESATU Bahwa ia Terdakwa...,52,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...,Kepolisian KESATU Bahwa ia Terdakwa I NYOMAN S...,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...,Seorang pria terdakwa seorang pria yang terdak...,DKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUART...,Bahwa selanjutnya dilakukan penimban...,dakwaan kesatu bahwa ia terdakwa i nyoman suar...,dakwaan kesatu bahwa ia terdakwa i nyoman suar...,dakwaan kesatu bahwa terdakwa i nyoman suarta ...,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...
3,DAKWAAN KESATU ...,72,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...,Kedakwaan KESATU Bahwa ia Terdakwa RICO JAYADI...,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...,"Seorang warga Banjar, RICOJAYADI, terdakwa RIC...",DKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI pa...,Plastik klip berisi butiran kristal ...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,dakwaan kesatu bahwa ia terdakwa rico jayadi p...,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...
4,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...,Ke Satu - Bahwa terdakwa SANG PUTU WIDIANA ber...,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU WID...,"Seorang terdakwa SANG PUTU WIDIANA, seorang sa...",Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU WID...,Bahwa terhadap barang bukti berupa 6...,dakwaan ke satu - satunya tentang terdakwa san...,dakwaan ke satu - satunya ditujukan terhadap t...,dakwaan terhadap terdakwa sang putu widiana be...,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...
...,...,...,...,...,...,...,...,...,...,...,...,...
1191,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12,Kesatu Pasal 83 ayat ( 1 ) huruf b UU RI No. 1...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18 ...,Kesatu Pasal 83 ayat 1 huruf b UU RI No. 18 ta...,Sebuah Pasal 83 ayat 1 huruf b UU RI No. 18 ta...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18 ...,NaN,kesatu pasal 83 ayat ( 1 ) huruf b uu ri no. 1...,dua pasal dalam uu ri nomor 18 tahun 2013 tent...,kesatu pasal 83 ayat ( 1 ) huruf b undang - un...,Kesatu Pasal 83 ayat(1) huruf b UU RI No. 18 t...
1192,Perbuatan terdakwa sebagaimana diatur da

# single model


In [ ]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("cahya/bert2bert-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2bert-indonesian-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 16  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],  # Include this line
                    min_length=batch_min_length,
                    max_length=batch_max_length,
                    num_beams=10,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95
                )
                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['cahya_bert2bert_summarization_dakwaan'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model {model_name} at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

Processing row 1/1196 - 100/1196


We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196


OSError: Cannot save file into a non-existent directory: '..\Data\STAGE_3_PREPROCESSING'

In [12]:
# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model cahya/bert2bert at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")


CHECKPOINT! Data Saved for model cahya/bert2bert at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_101016.csv
Summarization completed in 8207.40 seconds


In [ ]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("rowjak/bert-indonesian-news-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("rowjak/bert-indonesian-news-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 32  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],
                    num_beams=2,
                    repetition_penalty=2.5, 
                    length_penalty=1.0, 
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    max_length=batch_max_length,
                    min_length=batch_min_length)

                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['rowjak/bert-indonesian-news-summarization'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.bert.modeling_bert.BertLMHeadModel'> is overwritten by shared decoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "add_cross_attent

Processing row 1/1196 - 100/1196
Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196


NameError: name 'model_name' is not defined

In [14]:
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")


CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_105235.csv


In [15]:
import warnings
from transformers import BertTokenizer, EncoderDecoderModel
import time
import torch
import datetime
from torch.amp import autocast

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained("cahya/bert2gpt-indonesian-summarization")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token
model = EncoderDecoderModel.from_pretrained("cahya/bert2gpt-indonesian-summarization").to('cuda')

# Parameters
chunk_size = 450
batch_size = 32  # Number of chunks per batch for model inference

# To store summaries
summarized_texts = []

start_time = time.time()

# Iterate over rows
for index, row in df_extracted.iterrows():
    if index % 100 == 0:
        print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 100}/{len(df_extracted)}")
    text = row['cleaned_dakwaan']

    # Split the text into manageable chunks
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    # Summarize each batch of chunks
    summaries = []
    for i in range(0, len(chunks), batch_size):
        batch = chunks[i:i + batch_size]

        # Tokenize the batch
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=chunk_size).to('cuda')

        # Dynamic length adjustment
        valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
        if valid_lengths:
            batch_max_length = min(max(max(valid_lengths) // 3, 100), 200)  # Adjust max range
            batch_min_length = max(batch_max_length // 2, 50)  # Raise min range
        else:
            batch_max_length = 100  # Default max for very short chunks
            batch_min_length = 50  # Default min for very short chunks

        try:
            with autocast(device_type='cuda'):
            # Generate summaries
                summary_ids = model.generate(
                    input_ids=encoded_input['input_ids'],
                    attention_mask=encoded_input['attention_mask'],  # Include this line
                    min_length=batch_min_length,
                    max_length=batch_max_length,
                    num_beams=10,
                    repetition_penalty=2.5,
                    length_penalty=1.0,
                    early_stopping=True,
                    no_repeat_ngram_size=2,
                    use_cache=True,
                    do_sample=True,
                    temperature=0.8,
                    top_k=50,
                    top_p=0.95
                )
                summaries.extend([tokenizer.decode(ids, skip_special_tokens=True) for ids in summary_ids])
        except Exception as e:
            print(f"Error processing batch {i // batch_size}: {e}")
            summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

    # Combine all summaries for the current row
    # print(summaries)
    summarized_texts.append(" ".join(summaries))
    del encoded_input, summary_ids
    torch.cuda.empty_cache()

# Save summarized texts in the DataFrame
df_extracted['cahya_bert2gpt_summarization_dakwaan'] = summarized_texts

# Save intermediate results to avoid data loss
timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)
print(f"\nCHECKPOINT! Data Saved for model at {save_dir}")
print(f"Summarization completed in {time.time() - start_time:.2f} seconds")



Config of the encoder: <class 'transformers.models.bert.modeling_bert.BertModel'> is overwritten by shared encoder config: BertConfig {
  "_name_or_path": "cahya/bert-base-indonesian-1.5G",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.46.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

Config of the decoder: <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'> is overwritten by shared decoder config: GPT2Config {
  "_name_or_path": "cahya/gpt2-small-indonesian-522M",
  "activation_func

Processing row 1/1196 - 100/1196
Processing row 101/1196 - 200/1196
Processing row 201/1196 - 300/1196
Processing row 301/1196 - 400/1196
Processing row 401/1196 - 500/1196
Processing row 501/1196 - 600/1196
Processing row 601/1196 - 700/1196
Processing row 701/1196 - 800/1196
Processing row 801/1196 - 900/1196
Processing row 901/1196 - 1000/1196
Processing row 1001/1196 - 1100/1196
Processing row 1101/1196 - 1200/1196

CHECKPOINT! Data Saved for model at ../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_20241207_122142.csv
Summarization completed in 5332.07 seconds


In [27]:
del encoded_input
del summary_ids
del model
del tokenizer
torch.cuda.empty_cache()

NameError: name 'summary_ids' is not defined

In [ ]:
# dont run!

import warnings
from transformers import pipeline, AutoTokenizer
import time
import datetime
import torch
from torch.amp import autocast

chunk_size = 450
batch_size = 2048

for model_name in model_names:
    break
    pipelines = {}
    try:
        with warnings.catch_warnings(record=True) as caught_warnings:
            warnings.simplefilter("always")  # Capture all warnings
            tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

            # Check for the sentencepiece warning
            sentencepiece_warning = any(
                "The sentencepiece tokenizer that you are converting to a fast tokenizer" in str(w.message)
                for w in caught_warnings
            )

            if sentencepiece_warning:
                print(f"Warning for {model_name}: Sentencepiece tokenizer fallback detected. Using slow tokenizer.")
                use_fast = False
            else:
                use_fast = True

    except Exception as e:
        print(f"Fast tokenizer not supported for {model_name} due to error: {str(e)}. Using slow tokenizer.")
        use_fast = False

    # Create the summarization pipeline
    pipelines[model_name] = pipeline(
        "summarization",
        model=model_name,
        use_fast=use_fast,
        device=0,  # Change this to `-1` if not using GPU
    )

    for key, pipe in pipelines.items():
        start_time = time.time()
        summarized_texts = []  # To store summaries for each row

        for index, row in df_extracted.iterrows():
            # Print progress every 100 rows
            print(f"Processing row {index + 1}/{len(df_extracted)} for model {key}")
            # if index % 50 == 0:
                # print(f"Processing row {index + 1}/{len(df_extracted)} - {index + 50}/{len(df_extracted)} for model {key}")

            text = row['cleaned_dakwaan']

            # Split the text into manageable chunks
            chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

            # Summarize each batch of chunks
            summaries = []
            for i in range(0, len(chunks), batch_size):
                batch = chunks[i:i + batch_size]

                # Dynamically calculate max_length and min_length
                valid_lengths = [len(chunk.split()) for chunk in batch if chunk.strip()]
                if valid_lengths:
                    batch_max_length = max(min(max(length // 3 for length in valid_lengths), 150), 30)
                    batch_min_length = max(batch_max_length // 2, 20)
                else:
                    batch_max_length = 30
                    batch_min_length = 20

                try:
                    with autocast(device_type='cuda'):  # Enable mixed-precision for efficiency
                        results = pipe(
                            batch,
                            max_length=batch_max_length,
                            min_length=batch_min_length,
                            do_sample=False,
                        )
                        summaries.extend([result.get("generated_text", result.get("summary_text", "")) for result in results])
                except Exception as e:
                    print(f"Error with model {key} on batch {i // batch_size}: {e}")
                    summaries.extend(["" for _ in batch])  # Add empty summaries for failed batches

            # Combine all summaries for the current row
            summarized_texts.append(" ".join(summaries))

        # Save summarized texts in the DataFrame
        df_extracted[f'{key}_summarization_dakwaan'] = summarized_texts

        # Save intermediate results to avoid data loss
        timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        save_dir = f'../Data/STAGE_3_PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
        df_extracted.to_csv(save_dir, index=False)
        print(f"\nCHECKPOINT! Data Saved for model {key} at {save_dir}")

        # Log memory usage (assuming log_memory_usage is implemented elsewhere)
        # log_memory_usage()

        # Free GPU memory
        del pipes
        torch.cuda.empty_cache()
        print(f"Model {key} - Execution time: {time.time() - start_time:.2f} seconds\n")

    # Free pipelines dictionary
    del pipelines
    torch.cuda.empty_cache()


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Processing row 1/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 2/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 3/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 9. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=4)


Processing row 4/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


...

Your max_length is set to 30, but your input_length is only 15. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)


Processing row 24/1196 for model cahya/t5-base-indonesian-summarization-cased


Your max_length is set to 30, but your input_length is only 12. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)


Processing row 25/1196 for model cahya/t5-base-indonesian-summarization-cased
Processing row 26/1196 for model cahya/t5-base-indonesian-summarization-cased


: 

In [11]:
del pipe
torch.cuda.empty_cache()

In [12]:

del pipelines
torch.cuda.empty_cache()

In [24]:
save_dir = f'../Data/STAGE 3 PREPROCESSING/SINGARAJA_TEST_STAGE_3_PREPROCESSING_{timestamp}.csv'
df_extracted.to_csv(save_dir, index=False)

In [22]:
df_extracted

,klasifikasi_perkara,terdakwa,hakim,jumlah_saksi,cleaned_barang_bukti,cleaned_dakwaan,total_pidana_penjara_bulan,cahya/t5-base-indonesian-summarization-cased_summarization_dakwaan
0,Kejahatan Perjudian,KETUT SUBAGIA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
1,Kejahatan Perjudian,KOMANG ADITYA,I Made Bagiarta,3,1 (satu) buah papan bola bergambar 1 (satu) bu...,DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBA...,4,DAKWAAN PERTAMA- Bahwa Terdakwa KETUT SUBAGIA ...
2,Narkotika,I NYOMAN SUARTA,I Made Bagiarta,3,38 (tiga puluh delapan) potongan pipet plastik...,DAKWAAN KESATU Bahwa ia Terdakwa...,52,DAKWAAN KESATU Bahwa ia Terdakwa I NYOMAN SUAR...
3,Narkotika,RICO JAYADI,I Made Bagiarta,3,1 (satu) buah dompet warna hijau 20 (dua puluh...,DAKWAAN KESATU ...,72,DAKWAAN KESATU Bahwa ia Terdakwa RICO JAYADI p...
4,Narkotika,SANG PUTU WIDIANA,I Gusti Made Juliartawan,4,1(satu) buah HP merk Realme warna hitam putih,Dakwaan Ke Satu - Bahwa terdakwa SANG PUTU W...,48,Dakwaan Ke Satu- Bahwa terdakwa SANG PUTU WIDI...
...,...,...,...,...,...,...,...,...
1191,Perlindungan dan Pengelolaan Lingkungan Hidup,Jumat Ariyanto,Ida Bagus Bama Dewa. P,3,1 (satu) 1 (satu) batang kayu balang-balang be...,Kesatu Pasal 83 ayat (1) huruf b UU RI No. 18...,12,Kesatu Pasal 83 ayat(1) huruf b UU RI No. 18 t...
1192,Kejahatan Perjudian,I KETUT ANA SAPUTRA Alias BENCUT,Sudar,1,36 (tiga puluh enam) lembar kertas rekapan,Perbuatan terdakwa sebagaimana diatur dan dian...,4,Perbuatan terdakwa sebagaimana diatur dan dian...
1193,Penganiayaan,Sahabudin Alias Udin,Ni Luh Suantini,2,1. 1 (satu) buah tabung gas elpiji ukuran 3 (t...,Pasal 351 ayat (1) Kitab Undang-undang Hukum p...,9,Pasal 351 ayat( 1) Kitab Undang-undang Hukum p...
1194,Kejahatan yang Membahayakan Keamananan Umum Ba...,Putu Kristian Damanta,A.A. Sagung Yuni Wulantrisna,4,-1 (satu)unit sepeda motor merk Yamaha Jupiter...,Pertama Pasal 187 ayat (1) KUHP dan Kedua Pa...,24,Pasal 187 ayat(1) KUHP dan Kedua Pasal 363 aya...


In [ ]:
pipe = pipeline("text2text-generation", model="panggi/t5-small-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=2)

# Track the start time
start_time = time.time()

# Initialize an empty list to store the summaries
summarized_texts = []

# Iterate through the rows of the DataFrame
for index, row in df_extracted.iterrows():
    text = row['cleaned_dakwaan']

    # Chunk the text into smaller pieces
    chunk_size = 512
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]

    try:
        # Summarize each chunk
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    # Combine the summarized chunks into one string
    full_summary = " ".join(summaries)

    # Append the summarized text to the list
    summarized_texts.append(full_summary)

# Add the summarized texts as a new column
df_extracted['second_model_summarization_dakwaan'] = summarized_texts

# Print execution time
print(f"Execution time: {time.time() - start_time} seconds")


### DOCS

In [524]:
from transformers import pipeline

# pipe = pipeline("text2text-generation", model="facebook/mbart-large-50", use_fast=False, device="cuda", batch_size=2)
# pipe = pipeline("text2text-generation", model="indobenchmark/indobart-large")
# pipe = pipeline("summarization", model="mrm8488/t5-base-finetuned-indonesian-summarization")
# pipe = pipeline("text2text-generation", model="google/mt5-large", use_fast=False)

# tensorflow_summarization_models = [
#     "thonyyy/pegasus_indonesian_base-finetune",
#     "thonyyy/pegasus_indonesian_base-pretrain"
# ]

pytorch_summarization_models = [
    # "cahya/bert2bert-indonesian-summarization",
    "cahya/t5-base-indonesian-summarization-cased",
    "panggi/t5-small-indonesian-summarization-cased",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3",
    # "cahya/bert2gpt-indonesian-summarization",
    "panggi/t5-base-indonesian-summarization-cased",
    "rayendito/mt5-small-finetuned-xl-sum-indonesia",
    "interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs",
    # "rowjak/bert-indonesian-news-summarization"
]


# tensorflow_summarization_pipes = [
#     pipeline("summarization", model=model, from_tf=True, device=0, batch_size=4)
#     for model in tensorflow_summarization_models
# ]

In [ ]:
def evaluate_meteor(generated, reference):
    # Tokenize the generated and reference text
    tokenized_generated = word_tokenize(generated)
    tokenized_reference = word_tokenize(reference)
    
    # Evaluate METEOR score
    score = meteor_score.meteor_score([tokenized_reference], tokenized_generated)
    return score

def evaluate_rouge(generated, reference):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'], use_stemmer=True)
    scores = scorer.score(reference, generated)
    return scores

def evaluate_bleu(generated, reference):
    reference_tokens = reference.split()
    generated_tokens = generated.split()
    score = sentence_bleu([reference_tokens], generated_tokens)
    return score

# BERTScore evaluation
def evaluate_bertscore(generated, reference):
    P, R, F1 = score([generated], [reference], lang="en")
    return F1.item()

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Kalea\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
pipe = pipeline("summarization", model="cahya/bert2bert-indonesian-summarization", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="cahya/t5-base-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="panggi/t5-small-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-one-epoch", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-v3", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="cahya/bert2gpt-indonesian-summarization", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="panggi/t5-base-indonesian-summarization-cased", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="rayendito/mt5-small-finetuned-xl-sum-indonesia", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs", use_fast=False, device="cuda", batch_size=8)
pipe = pipeline("summarization", model="rowjak/bert-indonesian-news-summarization", use_fast=False, device="cuda", batch_size=8)


In [ ]:
import time

text = df_extracted.loc[0,'cleaned_dakwaan']
print(len(text))
reference_summary = """Terdakwa Ketut Subagia dan Komang Aditya pada 2 Juni 2024 sekitar pukul 00.30 WITA di pesisir pantai Gondol, Desa Penyabangan, Kecamatan Gerokgak, Kabupaten Buleleng, Bali, diduga mengadakan permainan judi bola adil secara ilegal. Ketut Subagia bertindak sebagai penyelenggara dan pemodal, sementara Komang Aditya membantu sebagai pekerja yang merapikan uang taruhan serta menyerahkannya kepada pemenang. Permainan melibatkan papan bergambar, bola karet, perlak bergambar, dan cek kayu sebagai pengganti uang taruhan. Barang bukti yang disita meliputi papan bola adil, tas, perlak, bola karet, kayu penyeimbang, bedak bayi, cek kayu dalam berbagai denominasi, lap, dan uang tunai Rp1.108.000. Permainan tersebut dilakukan dengan sistem taruhan yang memberikan hadiah kelipatan sembilan kali lipat dari jumlah taruhan jika menang, namun peserta yang kalah kehilangan uangnya. Aktivitas ini berlangsung selama beberapa jam dengan melibatkan sekitar 20 pemain dan keuntungan digunakan untuk kebutuhan sehari-hari. Namun, kegiatan ini dilakukan tanpa izin resmi dan melanggar hukum, sehingga kedua terdakwa ditangkap dan diancam pidana berdasarkan Pasal 303 KUHP juncto Undang-Undang Nomor 7 Tahun 1974 tentang Penertiban Perjudian."""

for i, model in enumerate(pytorch_summarization_models):
    print(model)
    pipe = pipeline("summarization", model=model, device="cuda", batch_size=4, use_fast=False) 
    chunk_size = 512 

    start_time = time.time()

    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    try:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['generated_text'] for chunk in chunks]
    except KeyError:
        summaries = [pipe(chunk, max_length=30, min_length=20, do_sample=False)[0]['summary_text'] for chunk in chunks]

    full_summary = " ".join(summaries)
    end_time = time.time()

    time_taken = end_time - start_time

    print("------------------------------------------------------")

    rouge_scores = evaluate_rouge(full_summary, reference_summary)
    print("ROUGE Scores:", rouge_scores)
    meteor_score_value = evaluate_meteor(full_summary, reference_summary)
    print("METEOR Score:", meteor_score_value)
    bleu_score_value = evaluate_bleu(full_summary, reference_summary)
    print("BLEU Score:", bleu_score_value)
    bertscore_value = evaluate_bertscore(full_summary, reference_summary)
    print("BERTScore F1:", bertscore_value)

    # Display ROUGE scores
    print(len(full_summary))
    print(f"Summary: using {model}: {full_summary}")
    print("------------------------------------------------------")

    del pipe
    torch.cuda.empty_cache()

    print(f"Time taken for summarization: {time_taken:.2f} seconds\n\n")


21579
cahya/t5-base-indonesian-summarization-cased


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.12121212121212122, recall=0.5847953216374269, fmeasure=0.20080321285140565), 'rouge2': Score(precision=0.04611650485436893, recall=0.2235294117647059, fmeasure=0.07645875251509053), 'rougeL': Score(precision=0.08242424242424243, recall=0.39766081871345027, fmeasure=0.13654618473895586), 'rougeLsum': Score(precision=0.08242424242424243, recall=0.39766081871345027, fmeasure=0.13654618473895586)}
METEOR Score: 0.30761823728726106
BLEU Score: 0.017997592188981122


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8316876292228699
5486
Summary: using cahya/t5-base-indonesian-summarization-cased: DAKWAAN PERTAMA - Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada saat terdakwa KETUT SUBAGIA bersama dengan Terdakwa UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Desa Penya Dua buah bola bergambar, satu (satu) buah tas kain warna abu-abu, satu (satu) buah tas karung warna Sebanyak 44 (empat puluh empat) buah papan cek kayu berwarna biru dengan nomor 20 (duapuluh), 17 (tujuh belas) KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola adil dan juga sebagai penyelenggara yang mengadakan permainan KOMANG ADITYA sebagai pekerja dan merapikan uang serta memberikan uang kepada pemenang permainan bola adil. Bahwa dalam permainan judi bola adil Pasangan uang taruhan judi bola adil 1 (satu)

...

BERTScore F1: 0.8033068180084229
176
Summary: using panggi/t5-base-indonesian-summarization-cased:                 Bahwa di papan bola adil terdapat 4 warna yaitu merah, kuning hijau dan hitam adapun gambarnya yaitu berbentuk segitiga, palang, bola,                          
------------------------------------------------------
Time taken for summarization: 17.71 seconds


rayendito/mt5-small-finetuned-xl-sum-indonesia
------------------------------------------------------
ROUGE Scores: {'rouge1': Score(precision=0.13314037626628075, recall=0.5380116959064327, fmeasure=0.2134570765661253), 'rouge2': Score(precision=0.043478260869565216, recall=0.17647058823529413, fmeasure=0.06976744186046512), 'rougeL': Score(precision=0.06801736613603473, recall=0.27485380116959063, fmeasure=0.10904872389791181), 'rougeLsum': Score(precision=0.06801736613603473, recall=0.27485380116959063, fmeasure=0.10904872389791181)}
METEOR Score: 0.3027963518759939
BLEU Score: 0.01799532323977946


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8162375688552856
4444
Summary: using rayendito/mt5-small-finetuned-xl-sum-indonesia: Seorang terdakwa kecelakaan pertama - telah mendakwa seorang terdakwa kecelakaan pertama - setelah Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan saksi Polisi dan Seorang pemain bola adil di Bali telah berlangsung permainan perjudian bola adil di pesisir pantai Gondol, Kec Sebuah tas kain warna hitam, 1 (satu) buah tas kain warna hitam, 1 (satu) buah tas Sebuah lembar uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari Seorang terdakwa keTUT SUBAGIA dalam permainan judi bola adil di Indonesia, telah mendakwa terdakwa keTUT Suba Pemenang permainan bola adil di Indonesia, telah menentukan kemenangan dalam permainan bola adil, karena seorang pemodal Permainan bola adil 1 (satu) buah tas karung warna biru garis merah digunakan untuk pasangan uang Sebuah papan cek kayu berwarna hitam digunakan untuk penyimpan bola adil setelah dig

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore F1: 0.8330612778663635
4361
Summary: using interstellarx95/mt5-small-finetuned-indonesian-text-summarization-three-epochs: Bahwa Terdakwa KETUT SUBAGIA bersama dengan Terdakwa KOMANG ADITYA pada hari Minggu tanggal 02 Juni 2024 udi bola adil dan menjadikan sebagai pencaharian, atau dengan sengaja turut serta dalam suatu perusahaan UT SUBAGIA dan Terdakwa KOMANG ADITYA sedang berlangsung permainan perjudian bola adil di pesisir pantai buah tas kain warna abu-abu, 1 (satu) buah perlak bergambar, 1 (satu) buah perlak uang pecahan Rp. 1.108.000,- (satu juta seratus delapan ribu rupiah) terdiri dari 8 (delapan Terdakwa KETUT SUBAGIA dalam permainan judi bola adil tersebut berperan sebagai penyelenggara yang mengadakan permainan judi bola Pemodal, mengambil uang - uang taruhan yang berada di atas perlak, menggelindingkan bola, sedangkan PAN bola adil 1 (satu) buah perlak bergambar digunakan untuk pasangan uang taruhan bola adil 1 (satu s kain warna hitam digunakan untuk penyimpan b